In [2]:
import shutil
from pathlib import Path

home = Path.home()

# Common cache locations for pybliometrics
cache_dirs = [
    home / ".pybliometrics" / "cache",
    home / ".pybliometrics" / "Scopus",
    home / ".cache" / "pybliometrics",
    home / ".cache" / "pybliometrics" / "Scopus",
]

for d in cache_dirs:
    if d.exists():
        print(f"Removing cache directory: {d}")
        shutil.rmtree(d)
    else:
        print(f"Not found: {d}")

print("✅ Pybliometrics cache cleared (wherever it was found).")



import pybliometrics
pybliometrics.scopus.init()


import pybliometrics

pybliometrics.scopus.utils.constants.CONFIG_FILE

Not found: C:\Users\yusef.atteyih\.pybliometrics\cache
Not found: C:\Users\yusef.atteyih\.pybliometrics\Scopus
Removing cache directory: C:\Users\yusef.atteyih\.cache\pybliometrics
Not found: C:\Users\yusef.atteyih\.cache\pybliometrics\Scopus
✅ Pybliometrics cache cleared (wherever it was found).


WindowsPath('C:/Users/yusef.atteyih/.config/pybliometrics.cfg')

In [33]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
r"""
Scopus → CiteScore with disambiguation by **Source ID then ASJC (sub-subject area)**

Fix in this version
• Resolved "unhashable type: 'set'" by storing ASJC sets as **frozenset** (hashable)
  and updating overlap logic to handle frozenset as well.
• Added an affiliation filter so only publications affiliated with a specific
  organization (affiliation ID) are collected.
• APP sheet now calculates Participation Score only for journal **articles**
  with Scopus subtype == "ar" (reviews 're' are excluded from APP).

What this script does
• Pulls publications for given Scopus Author IDs (AU-IDs) via pybliometrics
  but only those publications that list the configured `aff_id` as an affiliation.
• For each article/review, extracts: source_id, ISSN/e-ISSN, ASJC codes, etc.
• Loads your CiteScore 2024 table (supports CSV/XLSX; looks for columns:
  Print ISSN, E-ISSN, Percentile, CiteScore, and optionally Source ID + ASJC Code[s])
• Matches publications to CiteScore rows with the following priority:
    1) Same **Scopus source_id**; if multiple rows (different ASJC), pick the row
       whose **ASJC** overlaps the article’s ASJC codes; if still a tie, take the
       highest Percentile; if still tied, first occurrence.
    2) If no source_id match, try **ISSN / e-ISSN** with the same ASJC logic.
• Writes one Excel per author under ./authors, named like `_brahim_nalm___55537877400.xlsx`
   Each workbook contains:
     - "articles" sheet (all collected items that passed initial subtype filter)
     - "APP" sheet (Participation Score breakdown for last 3 calendar years, **articles only** subtype == "ar")

Usage notes
• By default this script filters publications by affiliation id `60021379`.
  You can override this with the CLI flag `--aff-id` or by passing `aff_id`
  into the `run(..., aff_id="...")` function.
"""

from __future__ import annotations

import argparse
import os
import re
import sys
import time
import warnings
from pathlib import Path
from typing import Any, Dict, Iterable, List, Optional, Sequence, Set, Tuple
from datetime import datetime

import pandas as pd

# --- Scopus (pybliometrics) ---
try:
    from pybliometrics.scopus import AbstractRetrieval, AuthorRetrieval, ScopusSearch, SerialTitle
    from pybliometrics.scopus.exception import ScopusException
except Exception as e:
    raise RuntimeError(
        "pybliometrics is required. Install with `pip install pybliometrics` "
        "and ensure your Scopus API credentials are configured."
    ) from e

# ----------------------- IPython detection ----------------------------

def _is_ipython() -> bool:
    return ("ipykernel" in sys.modules) or ("IPython" in sys.modules)

# ----------------------- Defaults & Affiliation filter ----------------

# Default affiliation id to filter publications by (Bahçeşehir University Scopus Affil ID)
AFF_ID_DEFAULT = "60021379"
# Active AFF_ID used by the script (can be overridden via CLI or run())
AFF_ID: Optional[str] = AFF_ID_DEFAULT

DEFAULT_USER_CITESCORE_DIRS = [
    Path(r"C:\\Users\\yusef.atteyih\\Desktop\\Academic Research Unit\\Yusef ATTEYIH\\Data Solutions\\Data Solutions 2.0\\APP Calculation\\CiteScore 2024"),
]
DEFAULT_CITESCORE_DIRS = [Path("CiteScore 2024"), Path(".")]
POSSIBLE_CS_FILENAMES = [
    "CiteScore 2024 annual values.csv",
    "CiteScore 2024 annual values.xlsx",
    "CiteScore 2024.csv",
    "citescore.csv",
    "citescore.xlsx",
]

# ----------------------- Small helpers --------------------------------

def _s(x) -> str:
    try:
        if x is None or (isinstance(x, float) and pd.isna(x)):
            return ""
    except Exception:
        pass
    return x if isinstance(x, str) else str(x)

def _norm_issn(s: str) -> str:
    """Uppercase; keep 0-9 and 'X'; strip others."""
    return re.sub(r"[^0-9X]", "", _s(s).upper())

def _norm_asjc_codes(raw: Any) -> Set[str]:
    """Return a set of 4-digit ASJC codes as strings."""
    out: Set[str] = set()
    if raw is None:
        return out
    if isinstance(raw, (list, tuple, set)):
        it = raw
    else:
        # split on non-digits; keep 4-digit tokens
        it = re.split(r"[^0-9]", _s(raw))
    for tok in it:
        if tok and tok.isdigit():
            if len(tok) == 4:
                out.add(tok)
            elif len(tok) > 4:
                # sometimes concatenated; take last 4 digits
                out.add(tok[-4:])
    return out

def _coerce_percentile(val) -> Optional[float]:
    txt = _s(val).strip()
    if not txt:
        return None
    txt = txt.replace("%", "").replace(" ", "").replace(",", ".")
    m = re.search(r"[-+]?\d*\.?\d+", txt)
    if not m:
        return None
    try:
        return float(m.group(0))
    except Exception:
        return None

def _coerce_float(val) -> Optional[float]:
    txt = _s(val).strip().replace(",", ".")
    if not txt:
        return None
    m = re.search(r"[-+]?\d*\.?\d+", txt)
    if not m:
        return None
    try:
        return float(m.group(0))
    except Exception:
        return None

def quartile_from_percentile(p: Optional[float]) -> str:
    if p is None:
        return ""
    try:
        p = float(p)
    except Exception:
        return ""
    if p >= 90: return "QT"
    if p >= 75: return "Q1"
    if p >= 50: return "Q2"
    if p >= 25: return "Q3"
    return "Q4"

# ----------------------- Filenames ------------------------------------

def _ascii_slug(s: str) -> str:
    import unicodedata
    s = unicodedata.normalize("NFKD", _s(s)).encode("ascii", "ignore").decode("ascii")
    s = s.lower()
    s = re.sub(r"[^0-9a-z]+", "_", s)
    s = re.sub(r"_+", "_", s).strip("_")
    return s

def make_author_filename(author_name: str, author_id: str) -> str:
    name_slug = _ascii_slug(author_name)
    leading = "_" + name_slug if not name_slug.startswith("_") else name_slug
    return f"{leading}___{author_id}.xlsx"

# ----------------------- Read AU-IDs ----------------------------------

author_id_file_candidates = ("authors.txt", "auids.txt", "ScopusAuthorIDs.txt")

def read_auids_from_cli_or_file(cli_auids: Optional[str]) -> List[str]:
    ids: List[str] = []
    if cli_auids:
        for tok in re.split(r"[,\s]+", cli_auids.strip()):
            if tok and tok.isdigit():
                ids.append(tok)
    if ids:
        return sorted(set(ids))
    for fname in author_id_file_candidates:
        p = Path(fname)
        if p.exists():
            try:
                raw = p.read_text(encoding="utf-8")
            except Exception:
                raw = p.read_text(errors="ignore")
            for line in raw.splitlines():
                t = line.split("#", 1)[0].strip()
                if t.isdigit():
                    ids.append(t)
            if ids:
                return sorted(set(ids))
    return []

# ----------------------- CiteScore path --------------------------------

def _candidate_files_in_dir(d: Path) -> List[Path]:
    return [p for name in POSSIBLE_CS_FILENAMES if (p := d / name).exists() and p.is_file()]

def resolve_citescore_path(arg: Optional[str], no_prompt: bool = False) -> Optional[Path]:
    if arg:
        p = Path(arg)
        if p.exists():
            if p.is_file(): return p
            if p.is_dir():
                cand = _candidate_files_in_dir(p)
                if cand: return cand[0]
    envp = os.environ.get("CITESCORE_CSV")
    if envp:
        p = Path(envp)
        if p.exists() and p.is_file(): return p
    for d in DEFAULT_USER_CITESCORE_DIRS + DEFAULT_CITESCORE_DIRS:
        if d.exists() and d.is_dir():
            cand = _candidate_files_in_dir(d)
            if cand: return cand[0]
    if not no_prompt:
        try:
            path_in = input("Path to CiteScore CSV/XLSX (or folder containing it): ").strip('"').strip()
            if path_in:
                p = Path(path_in)
                if p.exists():
                    if p.is_file(): return p
                    if p.is_dir():
                        cand = _candidate_files_in_dir(p)
                        if cand: return cand[0]
        except EOFError:
            pass
    return None

# ----------------------- Read CiteScore table --------------------------

def robust_read_table(path: Path) -> pd.DataFrame:
    if not path.exists():
        raise FileNotFoundError(f"CiteScore file not found: {path}")
    suffix = path.suffix.lower()
    if suffix in (".xlsx", ".xls"):
        return pd.read_excel(path)
    encodings = ["utf-8-sig", "utf-16", "utf-16le", "utf-16be", "cp1254", "iso-8859-9", "cp1252", "latin1"]
    last_err = None
    for enc in encodings:
        try:
            return pd.read_csv(path, encoding=enc, engine="python", sep=None)
        except Exception as e:
            last_err = e
            continue
    try:
        return pd.read_csv(path, encoding="latin1", engine="python", sep=None, on_bad_lines="skip")
    except TypeError:
        return pd.read_csv(path, encoding="latin1", engine="python", sep=None, error_bad_lines=False)  # type: ignore


def load_citescore_table(path: Path) -> pd.DataFrame:
    """
    Return columns:
      source_id (if present), issn_key, eissn_key, asjc_set (frozenset), cs_percentile, citescore
    Accepts flexible column names (case-insensitive)
    """
    cs = robust_read_table(path)
    norm = {c.strip().lower(): c for c in cs.columns}

    p = norm.get("print issn") or norm.get("p-issn") or norm.get("issn")
    e = norm.get("e-issn") or norm.get("eissn")
    pct = norm.get("percentile") or norm.get("citescore percentile")
    val = norm.get("citescore") or norm.get("citescore 2024")
    src = norm.get("source id") or norm.get("scopus source id") or norm.get("scopus sourceid")
    asjc_col = norm.get("asjc") or norm.get("asjc code") or norm.get("asjc codes") or norm.get("subject area asjc")

    if not all([p, e, pct, val]):
        raise KeyError(
            f"CiteScore table must include 'Print ISSN', 'E-ISSN', 'Percentile', 'CiteScore'. Found: {list(cs.columns)}"
        )

    # Standardize names
    cs = cs.rename(columns={p: "print_issn", e: "e_issn", pct: "cs_percentile", val: "citescore"})
    if src:
        cs = cs.rename(columns={src: "source_id"})
    if asjc_col:
        cs = cs.rename(columns={asjc_col: "asjc_raw"})
    else:
        cs["asjc_raw"] = ""

    # Coerce types
    cs["print_issn"] = cs["print_issn"].astype(str)
    cs["e_issn"] = cs["e_issn"].astype(str)
    cs["cs_percentile"] = cs["cs_percentile"].apply(_coerce_percentile)
    cs["citescore"] = cs["citescore"].apply(_coerce_float)
    if "source_id" in cs.columns:
        cs["source_id"] = cs["source_id"].astype(str).str.extract(r"(\d+)", expand=False).fillna("")

    # Normalized keys
    cs["issn_key"] = cs["print_issn"].map(_norm_issn)
    cs["eissn_key"] = cs["e_issn"].map(_norm_issn)

    # ASJC as frozenset (hashable)
    cs["asjc_set"] = cs["asjc_raw"].apply(lambda v: frozenset(_norm_asjc_codes(v)))

    # Keep minimal columns
    keep = ["issn_key", "eissn_key", "asjc_set", "cs_percentile", "citescore"]
    if "source_id" in cs.columns:
        keep.insert(0, "source_id")
    cs = cs[keep]
    return cs

# ----------------------- Scopus helpers --------------------------------

def _extract_issns(ar: Any) -> Tuple[str, str]:
    p = ""; e = ""
    if hasattr(ar, "eIssn"):
        e = _s(getattr(ar, "eIssn"))
    if not e and hasattr(ar, "e_issn"):
        e = _s(getattr(ar, "e_issn"))
    if hasattr(ar, "issn"):
        obj = getattr(ar, "issn")
        if isinstance(obj, str):
            if "ISSN(" in obj:
                mp = re.search(r"print\s*=\s*'([^']+)'", obj)
                me = re.search(r"electronic\s*=\s*'([^']+)'", obj)
                if mp: p = mp.group(1)
                if me and not e: e = me.group(1)
            else:
                p = _s(obj)
        else:
            try:
                p_obj = getattr(obj, "print", ""); e_obj = getattr(obj, "electronic", "")
                if p_obj: p = _s(p_obj)
                if not e and e_obj: e = _s(e_obj)
            except Exception:
                txt = _s(obj)
                mp = re.search(r"print\s*=\s*'([^']+)'", txt)
                me = re.search(r"electronic\s*=\s*'([^']+)'", txt)
                if mp: p = mp.group(1)
                if me and not e: e = me.group(1)
    return p, e


def _extract_asjc(ar: Any) -> Tuple[str, str, str, Set[str]]:
    codes: Set[str] = set(); areas: Set[str] = set(); abbrevs: Set[str] = set()
    sa = getattr(ar, "subject_areas", None)
    if sa:
        try:
            for it in sa:
                c = getattr(it, "code", None)
                a = getattr(it, "area", None)
                ab = getattr(it, "abbrev", None)
                if c is not None:
                    codes.add(f"{int(c):04d}" if str(c).isdigit() else str(c))
                if a: areas.add(_s(a))
                if ab: abbrevs.add(_s(ab))
        except Exception:
            try:  # dict-like
                for it in sa:
                    c = it.get("code"); a = it.get("area"); ab = it.get("abbrev")
                    if c is not None:
                        codes.add(f"{int(c):04d}" if str(c).isdigit() else str(c))
                    if a: areas.add(_s(a))
                    if ab: abbrevs.add(_s(ab))
            except Exception:
                pass
    return ", ".join(sorted(codes)), ", ".join(sorted(areas)), ", ".join(sorted(abbrevs)), codes

# ----------------------- ISSN → Source ID (optional) -------------------

def fetch_source_id_for_issn(issn: str) -> Optional[str]:
    issn = _norm_issn(issn)
    if not issn:
        return None
    try:
        res = SerialTitle(issn)
        items: Iterable[Any]
        try:
            items = list(res) if isinstance(res, (list, tuple)) else [res]
        except Exception:
            items = [res]
        for it in items:
            for attr in ("source_id", "sourcerecord_id", "sourceid"):
                if hasattr(it, attr):
                    sid = _s(getattr(it, attr))
                    if sid.isdigit():
                        return sid
        return None
    except Exception as e:
        warnings.warn(f"SerialTitle lookup failed for ISSN {issn}: {e}")
        return None


def build_cs_by_source(cs_table: pd.DataFrame, serial_sleep: float = 0.1) -> pd.DataFrame:
    """Ensure a cs_by_source table with 'source_id' present, mapping from cs_table.
    If cs_table already has source_id, just dedupe and return.
    Otherwise, look up source_id from ISSN/e-ISSN via SerialTitle.
    """
    if "source_id" in cs_table.columns:
        df = cs_table.copy()
    else:
        df = cs_table.copy()
        # NOTE: These are vectorized calls; add throttling inside fetch if needed.
        df["source_id_from_print"] = df["issn_key"].apply(fetch_source_id_for_issn)
        if serial_sleep:
            time.sleep(serial_sleep)
        df["source_id_from_e"] = df["eissn_key"].apply(fetch_source_id_for_issn)
        df["source_id"] = df["source_id_from_print"].where(df["source_id_from_print"].notna(), df["source_id_from_e"])
        df = df.drop(columns=["source_id_from_print", "source_id_from_e"], errors="ignore")
    # Keep only rows with a source_id
    df = df[df["source_id"].astype(str).str.strip().ne("")].copy()
    # Deduplicate by (source_id, asjc_set); asjc_set is frozenset → hashable
    df = df.drop_duplicates(subset=["source_id", "asjc_set"], keep="first")
    return df[["source_id", "asjc_set", "cs_percentile", "citescore"]]

# ----------------------- Matching logic --------------------------------

def _pick_best_candidate(cands: pd.DataFrame, article_asjc: Set[str]) -> Tuple[Optional[float], Optional[float]]:
    """Choose best row: prefer ASJC overlap, then highest percentile, else first.
    Returns (cs_percentile, citescore).
    """
    if cands is None or cands.empty:
        return None, None
    # compute overlap count
    over = []
    for _i, row in cands.iterrows():
        cs_set = row.get("asjc_set") or set()
        # Accept frozenset/list/tuple/set/string
        if isinstance(cs_set, (set, frozenset)):
            cs_set2 = set(cs_set)
        elif isinstance(cs_set, (list, tuple)):
            cs_set2 = {str(x) for x in cs_set}
        else:
            cs_set2 = _norm_asjc_codes(cs_set)
        over.append(len(article_asjc & cs_set2))
    cands = cands.copy()
    cands["_overlap"] = over
    # sort by: overlap desc, percentile desc (None last)
    cands["_pct"] = cands["cs_percentile"].fillna(-1e9)
    cands = cands.sort_values(["_overlap", "_pct"], ascending=[False, False])
    top = cands.iloc[0]
    return top.get("cs_percentile"), top.get("citescore")


def enrich_with_citescore_sourceid_asjc(
    df_articles: pd.DataFrame,
    cs_table: pd.DataFrame,
    cs_by_source: pd.DataFrame,
) -> pd.DataFrame:
    if df_articles is None or df_articles.empty:
        return df_articles.copy()

    df = df_articles.copy()
    for col in ("issn_print", "issn_electronic", "source_id"):
        if col not in df.columns:
            df[col] = ""

    # Prepare article keys
    df["issn_key"] = df["issn_print"].astype(str).map(_norm_issn)
    df["eissn_key"] = df["issn_electronic"].astype(str).map(_norm_issn)

    # Pre-split article ASJC sets
    a_asjc_sets: List[Set[str]] = []
    for v in df.get("asjc_codes", pd.Series([""] * len(df))):
        a_asjc_sets.append(_norm_asjc_codes(v))

    cs_p = cs_table[["issn_key", "asjc_set", "cs_percentile", "citescore"]].drop_duplicates()
    cs_e = cs_table[["eissn_key", "asjc_set", "cs_percentile", "citescore"]].drop_duplicates()

    out_pct: List[Optional[float]] = []
    out_val: List[Optional[float]] = []

    for idx, row in df.iterrows():
        article_asjc = a_asjc_sets[idx] if idx < len(a_asjc_sets) else set()
        sid = _s(row.get("source_id"))
        pct = None; val = None
        # 1) try source_id
        if sid:
            cands = cs_by_source[cs_by_source["source_id"].astype(str) == sid]
            pct, val = _pick_best_candidate(cands, article_asjc)
        # 2) fallback by ISSN
        if pct is None and val is None:
            issn = _s(row.get("issn_key"))
            eissn = _s(row.get("eissn_key"))
            cands = pd.concat([
                cs_p[cs_p["issn_key"] == issn],
                cs_e[cs_e["eissn_key"] == eissn],
            ], ignore_index=True)
            pct, val = _pick_best_candidate(cands, article_asjc)
        out_pct.append(pct)
        out_val.append(val)

    df["cs_percentile"] = out_pct
    df["citescore"] = out_val
    df["quartile"] = df["cs_percentile"].apply(quartile_from_percentile)
    return df

# ----------------------- Scopus collectors (AFF filter applied) -------

def get_author_name(author_id: str) -> str:
    try:
        ar = AuthorRetrieval(author_id)
        name = f"{_s(ar.given_name)} {_s(ar.surname)}".strip()
        return name or author_id
    except Exception:
        return author_id


def get_author_eids(author_id: str) -> List[str]:
    """
    Return EIDs for the given author that are affiliated with AFF_ID (if set).
    Query uses AU-ID(...) AND AF-ID(<aff_id>) when AFF_ID is provided.
    """
    query = f"AU-ID({author_id})"
    if AFF_ID:
        # AF-ID filters by organization affiliation id in Scopus
        query = f"{query} AND AF-ID({AFF_ID})"
    s = ScopusSearch(query, subscriber=True)
    return s.get_eids() or []


def get_article_metadata(eid: str):
    try:
        ar = AbstractRetrieval(eid, view="FULL")
    except ScopusException as e:
        warnings.warn(f"AbstractRetrieval failed for {eid}: {e}")
        return None

    # Keep both articles and reviews in the collected list, but APP will count only subtype "ar"
    if ar.subtype not in ("ar", "re"):
        return None

    year = ""
    if getattr(ar, "coverDate", None):
        year = _s(ar.coverDate)[:4]

    issn_print, issn_elec = _extract_issns(ar)
    asjc_codes_csv, asjc_areas_csv, asjc_abbrevs_csv, _codes_set = _extract_asjc(ar)

    return {
        "eid": _s(eid),
        "title": _s(ar.title),
        "year": _s(year),
        "publication_name": _s(getattr(ar, "publicationName", "")),
        "subtype": _s(ar.subtype),
        "doi": _s(getattr(ar, "doi", "")),
        "source_id": _s(getattr(ar, "source_id", "")),
        "issn_print": _s(issn_print),
        "issn_electronic": _s(issn_elec),
        "asjc_codes": asjc_codes_csv,
        "asjc_areas": asjc_areas_csv,
        "asjc_abbrevs": asjc_abbrevs_csv,
        "authors_count": len(ar.authors) if getattr(ar, "authors", None) else 1,
        "combined": "; ".join([t for t in (getattr(ar, "authkeywords", []) or []) if _s(t)]),
        "abstract": _s(getattr(ar, "description", "")),
    }

# ----------------------- APP calculation helpers -----------------------

def _qc_from_percentile(p: Optional[float]) -> Optional[float]:
    """QC mapping assuming higher percentile is better (>=90 is Top 10%)."""
    if p is None:
        return None
    try:
        p = float(p)
    except Exception:
        return None
    if p >= 90:   # Top 10%
        return 1.4
    if p >= 75:   # Q1
        return 1.0
    if p >= 50:   # Q2
        return 0.8
    if p >= 25:   # Q3
        return 0.6
    if p >= 0:    # Q4
        return 0.4
    return None

def _ac_from_authors(n: Any) -> float:
    """Author Coefficient: 1.2 if single author else 1.2 / n_authors."""
    try:
        n = int(n)
    except Exception:
        n = 1
    return 1.2 if n <= 1 else 1.2 / max(n, 1)

def _to_int_year(y: Any) -> Optional[int]:
    try:
        s = str(y)
        m = re.search(r"\d{4}", s)
        return int(m.group(0)) if m else None
    except Exception:
        return None

def build_app_sheet(df_articles: pd.DataFrame, now_year: Optional[int] = None) -> Tuple[pd.DataFrame, Dict[str, Any]]:
    """
    Returns (df_app, summary) where df_app contains the per-paper APP breakdown and
    summary has 'app_total' and 'eligibility' text.

    NOTE: APP is calculated only for journal articles with subtype == "ar".
    """
    if df_articles is None or df_articles.empty:
        return pd.DataFrame(), {"app_total": 0.0, "eligibility": "No eligible items", "years": []}

    # Determine the 3-year window (includes current year)
    cy = now_year or datetime.now().year
    years_ok = {cy, cy - 1, cy - 2}

    # Coerce needed fields
    tmp = df_articles.copy()
    tmp["year_i"] = tmp.get("year", "").apply(_to_int_year)
    tmp["cs_percentile_num"] = pd.to_numeric(tmp.get("cs_percentile"), errors="coerce")
    tmp["authors_count_i"] = pd.to_numeric(tmp.get("authors_count"), errors="coerce").fillna(1).astype(int)
    tmp["subtype_norm"] = tmp.get("subtype", "").astype(str).str.lower()

    # Eligibility: last 3 calendar years, subtype == "ar" only, has percentile
    eligible = tmp[
        tmp["year_i"].isin(years_ok) &
        tmp["cs_percentile_num"].notna() &
        (tmp["subtype_norm"] == "ar")
    ].copy()

    if eligible.empty:
        return pd.DataFrame(), {"app_total": 0.0, "eligibility": "No eligible items (no 'ar' articles in window)", "years": sorted(years_ok)}

    # Compute QC, AC, Contribution
    eligible["QC"] = eligible["cs_percentile_num"].apply(_qc_from_percentile)
    eligible["AC"] = eligible["authors_count_i"].apply(_ac_from_authors)
    # If QC missing after mapping, treat as ineligible
    eligible = eligible[eligible["QC"].notna()].copy()
    if eligible.empty:
        return pd.DataFrame(), {"app_total": 0.0, "eligibility": "No eligible items (QC missing)", "years": sorted(years_ok)}

    eligible["Contribution"] = eligible["AC"] * eligible["QC"]

    # Round to 2 decimals per policy
    eligible["AC"] = eligible["AC"].round(2)
    eligible["QC"] = eligible["QC"].round(2)
    eligible["Contribution"] = eligible["Contribution"].round(2)

    # Select user-friendly columns
    out_cols = [
        "eid", "title", "year", "publication_name",
        "authors_count", "cs_percentile", "quartile",
        "AC", "QC", "Contribution"
    ]
    for c in out_cols:
        if c not in eligible.columns:
            eligible[c] = pd.Series(dtype="object")
    df_app = eligible[out_cols].sort_values(["year", "Contribution"], ascending=[False, False]).reset_index(drop=True)

    app_total = float(df_app["Contribution"].sum().round(2))

    # Eligibility band (based on your table)
    if app_total > 1.0:
        elig = "APP > 1.0 → up to 2 supports / AY (only 1 requires full indexing & APP check)"
    elif app_total >= 0.4:
        elig = "0.4 ≤ APP ≤ 1.0 → 1 support / AY"
    else:
        elig = "APP < 0.4 → 1 support / AY (if other criteria met)"

    summary = {"app_total": round(app_total, 2), "eligibility": elig, "years": sorted(years_ok)}
    return df_app, summary

# ----------------------- Excel writer (with APP sheet) -----------------

def _write_excel_xlsxwriter(df: pd.DataFrame, path: Path, app_df: Optional[pd.DataFrame] = None, app_summary: Optional[Dict[str, Any]] = None):
    with pd.ExcelWriter(path, engine="xlsxwriter") as xl:
        df.to_excel(xl, sheet_name="articles", index=False)
        ws = xl.sheets["articles"]
        try:
            ws.freeze_panes(1, 0)
            ws.set_column("A:A", 20)  # eid
            ws.set_column("B:B", 50)  # title
            ws.set_column("C:C", 8)   # year
            ws.set_column("D:D", 36)  # publication_name
            ws.set_column("E:E", 8)   # subtype
            ws.set_column("F:F", 26)  # doi
            ws.set_column("G:G", 14)  # source_id
            ws.set_column("H:I", 14)  # ISSNs
            ws.set_column("J:J", 18)  # asjc_codes
            ws.set_column("K:K", 26)  # asjc_areas
            ws.set_column("L:L", 14)  # asjc_abbrevs
            ws.set_column("M:N", 14)  # cs_percentile / citescore
            ws.set_column("O:O", 10)  # quartile
            ws.set_column("P:P", 12)  # authors_count
            ws.set_column("Q:Q", 28)  # combined
            ws.set_column("R:R", 80)  # abstract
        except Exception:
            pass

        # Sheet 2: APP (optional)
        if app_df is not None and not app_df.empty:
            # write header and summary first (we'll write table starting at row 6)
            ws2 = xl.book.add_worksheet("APP")
            xl.sheets["APP"] = ws2  # make sure mapping exists
            try:
                ws2.write(0, 0, "APP calculation — last 3 calendar years (journal articles, subtype == 'ar')")
                if app_summary:
                    ws2.write(1, 0, "Years considered")
                    ws2.write(1, 1, ", ".join(str(y) for y in app_summary.get("years", [])))
                    ws2.write(2, 0, "APP Score")
                    ws2.write(2, 1, app_summary.get("app_total", 0.0))
                    ws2.write(3, 0, "Eligibility")
                    ws2.write(3, 1, app_summary.get("eligibility", ""))
                # write the dataframe starting at row 6 (index 5) to leave space for header
                (app_df.reset_index(drop=True)).to_excel(xl, sheet_name="APP", index=False, startrow=5)
                # adjust columns
                ws2.freeze_panes(6, 0)
                ws2.set_column("A:A", 20)  # eid
                ws2.set_column("B:B", 60)  # title
                ws2.set_column("C:C", 8)   # year
                ws2.set_column("D:D", 36)  # journal
                ws2.set_column("E:E", 12)  # authors_count
                ws2.set_column("F:F", 14)  # cs_percentile
                ws2.set_column("G:G", 10)  # quartile
                ws2.set_column("H:J", 14)  # AC, QC, Contribution
            except Exception:
                # Fallback: if something goes wrong with custom worksheet, write simply by pandas
                try:
                    app_df.to_excel(xl, sheet_name="APP", index=False)
                except Exception:
                    pass

def _write_excel_openpyxl(df: pd.DataFrame, path: Path, app_df: Optional[pd.DataFrame] = None, app_summary: Optional[Dict[str, Any]] = None):
    with pd.ExcelWriter(path, engine="openpyxl") as xl:
        df.to_excel(xl, sheet_name="articles", index=False)
        try:
            ws = xl.sheets["articles"]
            ws.freeze_panes = "A2"
        except Exception:
            pass
        if app_df is not None and not app_df.empty:
            try:
                app_df.to_excel(xl, sheet_name="APP", index=False, startrow=5)
                ws2 = xl.sheets.get("APP", None)
                if ws2 is not None:
                    ws2.freeze_panes = "A6"
            except Exception:
                try:
                    app_df.to_excel(xl, sheet_name="APP", index=False)
                except Exception:
                    pass

def save_author_excel(author_id: str, author_name: str, df_articles: pd.DataFrame, out_dir: Path) -> str:
    out_dir.mkdir(parents=True, exist_ok=True)
    path = out_dir / make_author_filename(author_name, author_id)
    need = [
        "eid","title","year","publication_name","subtype","doi",
        "source_id","issn_print","issn_electronic",
        "asjc_codes","asjc_areas","asjc_abbrevs",
        "cs_percentile","citescore","quartile",
        "authors_count","combined","abstract"
    ]
    df = df_articles.copy()
    for c in need:
        if c not in df.columns:
            df[c] = pd.Series(dtype="object")
    df = df[need]

    # Build APP breakdown & summary
    app_df, app_summary = build_app_sheet(df)

    try:
        _write_excel_xlsxwriter(df, path, app_df=app_df if not app_df.empty else None, app_summary=app_summary)
    except Exception:
        _write_excel_openpyxl(df, path, app_df=app_df if not app_df.empty else None, app_summary=app_summary)
    return str(path)

# ----------------------- Orchestration --------------------------------

def process_author(author_id: str, cs_table: pd.DataFrame, out_dir: Path, sleep: float, serial_sleep: float, cs_by_source: Optional[pd.DataFrame] = None) -> Optional[str]:
    try:
        name = get_author_name(author_id)
        print(f"→ AU-ID {author_id} — {name} (filtering by AFF_ID={AFF_ID})")
        eids = get_author_eids(author_id)
        recs: List[Dict[str, Any]] = []
        for eid in eids:
            md = get_article_metadata(eid)
            if md:
                recs.append(md)
            if sleep:
                time.sleep(sleep)
        df = pd.DataFrame(recs)

        if not df.empty:
            # build cs_by_source if not provided (backwards-compatible)
            if cs_by_source is None:
                cs_by_source = build_cs_by_source(cs_table, serial_sleep=serial_sleep)
            df = enrich_with_citescore_sourceid_asjc(df, cs_table, cs_by_source)

        path = save_author_excel(author_id, name, df if not df.empty else pd.DataFrame(), out_dir)
        print(f"   ✓ Wrote {Path(path).name}  ({0 if df is None else len(df)} rows)")
        return path
    except Exception as e:
        print(f"   ⚠️ Failed {author_id}: {e}")
        return None

# ----------------------- Public API (Jupyter) -------------------------

def run(
    auids: Optional[str] = None,
    citescore: Optional[str] = None,
    outdir: str = "authors",
    sleep: float = 0.05,
    serial_sleep: float = 0.1,
    no_prompt: bool = True,
    aff_id: Optional[str] = None,
) -> None:
    """
    Jupyter-friendly entrypoint. Pass aff_id to override the default AFF_ID filter.
    """
    global AFF_ID
    if aff_id:
        AFF_ID = aff_id

    out_dir = Path(outdir)
    cs_path = resolve_citescore_path(citescore, no_prompt=no_prompt)
    if not cs_path:
        raise FileNotFoundError(
            "Could not locate CiteScore file. Pass citescore=..., set CITESCORE_CSV env var, or place the file in a default folder."
        )
    print(f"Using CiteScore file: {cs_path}")
    cs_table = load_citescore_table(cs_path)

    ids = read_auids_from_cli_or_file(auids)
    if not ids:
        print("No AU-IDs provided. Provide auids='555...,572...' or authors.txt file.")
        return
    print(f"CiteScore rows: {len(cs_table)} | Authors: {len(ids)} | Affiliation filter: {AFF_ID}")

    # Build cs_by_source once per run to avoid repeated SerialTitle lookups
    cs_by_source = build_cs_by_source(cs_table, serial_sleep=serial_sleep)

    out_dir.mkdir(parents=True, exist_ok=True)
    for i, auid in enumerate(ids, 1):
        print(f"[{i}/{len(ids)}]")
        process_author(auid, cs_table, out_dir, sleep=sleep, serial_sleep=serial_sleep, cs_by_source=cs_by_source)
    print(f"\nDone. Files saved to: {out_dir.resolve()}")

# ----------------------- CLI -----------------------------------------

def main(argv: Optional[list] = None):
    global AFF_ID
    ap = argparse.ArgumentParser(description="Fetch Scopus pubs by AU-ID and match CiteScore by Source ID then ASJC (ISSN fallback).")
    ap.add_argument("--auids", type=str, default=None, help="Comma/space-separated Scopus Author IDs. If omitted, reads authors.txt")
    ap.add_argument("--citescore", type=str, default=None, help="Path to CiteScore CSV/XLSX, or a folder containing it")
    ap.add_argument("--outdir", type=str, default="authors", help="Output directory (default: ./authors)")
    ap.add_argument("--sleep", type=float, default=0.05, help="Sleep between EID fetches (seconds)")
    ap.add_argument("--serial-sleep", type=float, default=0.1, help="Sleep between SerialTitle ISSN lookups (seconds)")
    ap.add_argument("--no-prompt", action="store_true", help="Do not prompt for missing CiteScore path; exit with error")
    ap.add_argument("--aff-id", type=str, default=None, help=f"Optional affiliation ID to filter publications by (default: {AFF_ID_DEFAULT})")
    args, _unknown = ap.parse_known_args(argv)

    if args.aff_id:
        AFF_ID = args.aff_id

    cs_path = resolve_citescore_path(args.citescore, no_prompt=args.no_prompt)
    if not cs_path:
        if _is_ipython() and not args.no_prompt:
            try:
                entered = input("Enter CiteScore CSV/XLSX path or containing folder: ").strip('"').strip()
            except EOFError:
                entered = ""
            if entered:
                cs_path = resolve_citescore_path(entered, no_prompt=True)
        if not cs_path:
            print("❌ Could not locate the CiteScore file. Pass --citescore or set CITESCORE_CSV.")
            if _is_ipython():
                return
            sys.exit(2)
    print(f"Using CiteScore file: {cs_path}")
    cs_table = load_citescore_table(cs_path)

    auids = read_auids_from_cli_or_file(args.auids)
    if not auids and _is_ipython():
        try:
            entered = input("Enter Scopus Author IDs (comma or space separated), or leave blank to cancel: ").strip()
        except EOFError:
            entered = ""
        if entered:
            auids = read_auids_from_cli_or_file(entered)
    if not auids:
        print("No AU-IDs provided. Use --auids, or create authors.txt with one AU-ID per line.")
        if _is_ipython():
            return
        sys.exit(0)

    out_dir = Path(args.outdir)
    print(f"CiteScore rows: {len(cs_table)} | Authors: {len(auids)} | Out: {out_dir.resolve()} | Affiliation filter: {AFF_ID}")
    out_dir.mkdir(parents=True, exist_ok=True)

    # Build cs_by_source once for the whole run
    cs_by_source = build_cs_by_source(cs_table, serial_sleep=args.serial_sleep)

    written = []
    for i, auid in enumerate(auids, 1):
        print(f"[{i}/{len(auids)}] Processing AU-ID {auid} … (AFF_ID={AFF_ID})")
        p = process_author(auid, cs_table, out_dir, sleep=args.sleep, serial_sleep=args.serial_sleep, cs_by_source=cs_by_source)
        if p:
            written.append(p)

    print(f"\nDone. Wrote {len(written)} file(s) to {out_dir.resolve()}.")

if __name__ == "__main__":
    main()


Using CiteScore file: C:\Users\yusef.atteyih\Desktop\Academic Research Unit\Yusef ATTEYIH\Data Solutions\Data Solutions 2.0\APP Calculation\CiteScore 2024\CiteScore 2024 annual values.csv


Enter Scopus Author IDs (comma or space separated), or leave blank to cancel:  57193254610


CiteScore rows: 75679 | Authors: 1 | Out: C:\Users\yusef.atteyih\Desktop\Academic Research Unit\Yusef ATTEYIH\Data Solutions\Data Solutions 2.0\APP Calculation\authors | Affiliation filter: 60021379
[1/1] Processing AU-ID 57193254610 … (AFF_ID=60021379)
→ AU-ID 57193254610 — Fadime İrem Doğan (filtering by AFF_ID=60021379)
   ✓ Wrote _fadime_irem_dogan___57193254610.xlsx  (3 rows)

Done. Wrote 1 file(s) to C:\Users\yusef.atteyih\Desktop\Academic Research Unit\Yusef ATTEYIH\Data Solutions\Data Solutions 2.0\APP Calculation\authors.


In [35]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
r"""
Scopus → CiteScore with disambiguation by **Source ID then ASJC (sub-subject area)**

This version:
• Stores ASJC sets as frozenset (hashable).
• Matches publications to CiteScore by Source ID then ASJC (ISSN fallback).
• Adds an affiliation-*detection and isolation* flow equivalent to the
  logic used in the longer ETL script: per-article author→affiliation mapping
  is parsed from the Scopus AbstractRetrieval results and each article is
  kept for an author only if that author appears on the article *and* is
  affiliated with the configured AFF_ID on that article.
• Adds an "APP" sheet to the output Excel workbook (Participation Score
  calculation). APP counts only journal **articles** (subtype == "ar")
  within the last three calendar years (current year, -1, -2).
• APP uses Author Coefficient (AC = 1.2 if sole author else 1.2 / n_authors)
  and Quartile Coefficient (QC) mapped from CiteScore percentile,
  Contribution = AC × QC, summed and rounded.

Usage:
  - In Jupyter: call run(auids="12345,67890", citescore="path/to/CiteScore.csv", aff_id="60021379")
  - CLI: python this_script.py --auids 57193254610 --citescore "CiteScore 2024.csv" --aff-id 60021379

Requirements:
  pybliometrics, pandas, xlsxwriter or openpyxl
  Configure pybliometrics Scopus API credentials
"""

from __future__ import annotations

import argparse
import glob
import json
import os
import re
import sys
import time
import unicodedata
import warnings
from datetime import datetime
from pathlib import Path
from typing import Any, Dict, Iterable, List, Optional, Set, Tuple

import pandas as pd

# --- Scopus (pybliometrics) ---
try:
    from pybliometrics.scopus import AbstractRetrieval, AuthorRetrieval, ScopusSearch, SerialTitle
    from pybliometrics.scopus.exception import ScopusException
except Exception as e:
    raise RuntimeError(
        "pybliometrics is required. Install with `pip install pybliometrics` "
        "and ensure your Scopus API credentials are configured."
    ) from e

# ----------------------- IPython detection ----------------------------

def _is_ipython() -> bool:
    return ("ipykernel" in sys.modules) or ("IPython" in sys.modules)

# ----------------------- Defaults & Affiliation filter ----------------

# Default affiliation id to filter publications by (Bahçeşehir University Scopus Affil ID)
AFF_ID_DEFAULT = "60021379"
AFF_ID: Optional[str] = AFF_ID_DEFAULT

DEFAULT_USER_CITESCORE_DIRS = [
    Path(r"C:\\Users\\yusef.atteyih\\Desktop\\Academic Research Unit\\Yusef ATTEYIH\\Data Solutions\\Data Solutions 2.0\\APP Calculation\\CiteScore 2024"),
]
DEFAULT_CITESCORE_DIRS = [Path("CiteScore 2024"), Path(".")]
POSSIBLE_CS_FILENAMES = [
    "CiteScore 2024 annual values.csv",
    "CiteScore 2024 annual values.xlsx",
    "CiteScore 2024.csv",
    "citescore.csv",
    "citescore.xlsx",
]

# ----------------------- Small helpers --------------------------------

def _s(x) -> str:
    try:
        if x is None or (isinstance(x, float) and pd.isna(x)):
            return ""
    except Exception:
        pass
    return x if isinstance(x, str) else str(x)

def _norm_issn(s: str) -> str:
    """Uppercase; keep 0-9 and 'X'; strip others."""
    return re.sub(r"[^0-9X]", "", _s(s).upper())

def _norm_asjc_codes(raw: Any) -> Set[str]:
    """Return a set of 4-digit ASJC codes as strings."""
    out: Set[str] = set()
    if raw is None:
        return out
    if isinstance(raw, (list, tuple, set)):
        it = raw
    else:
        # split on non-digits; keep 4-digit tokens
        it = re.split(r"[^0-9]", _s(raw))
    for tok in it:
        if tok and tok.isdigit():
            if len(tok) == 4:
                out.add(tok)
            elif len(tok) > 4:
                # sometimes concatenated; take last 4 digits
                out.add(tok[-4:])
    return out

def _coerce_percentile(val) -> Optional[float]:
    txt = _s(val).strip()
    if not txt:
        return None
    txt = txt.replace("%", "").replace(" ", "").replace(",", ".")
    m = re.search(r"[-+]?\d*\.?\d+", txt)
    if not m:
        return None
    try:
        return float(m.group(0))
    except Exception:
        return None

def _coerce_float(val) -> Optional[float]:
    txt = _s(val).strip().replace(",", ".")
    if not txt:
        return None
    m = re.search(r"[-+]?\d*\.?\d+", txt)
    if not m:
        return None
    try:
        return float(m.group(0))
    except Exception:
        return None

def quartile_from_percentile(p: Optional[float]) -> str:
    if p is None:
        return ""
    try:
        p = float(p)
    except Exception:
        return ""
    if p >= 90: return "QT"
    if p >= 75: return "Q1"
    if p >= 50: return "Q2"
    if p >= 25: return "Q3"
    return "Q4"

# ----------------------- Filenames ------------------------------------

def _ascii_slug(s: str) -> str:
    s = unicodedata.normalize("NFKD", _s(s)).encode("ascii", "ignore").decode("ascii")
    s = s.lower()
    s = re.sub(r"[^0-9a-z]+", "_", s)
    s = re.sub(r"_+", "_", s).strip("_")
    return s

def make_author_filename(author_name: str, author_id: str) -> str:
    name_slug = _ascii_slug(author_name)
    leading = "_" + name_slug if not name_slug.startswith("_") else name_slug
    return f"{leading}___{author_id}.xlsx"

# ----------------------- Read AU-IDs ----------------------------------

author_id_file_candidates = ("authors.txt", "auids.txt", "ScopusAuthorIDs.txt")

def read_auids_from_cli_or_file(cli_auids: Optional[str]) -> List[str]:
    ids: List[str] = []
    if cli_auids:
        for tok in re.split(r"[,\s]+", cli_auids.strip()):
            if tok and tok.isdigit():
                ids.append(tok)
    if ids:
        return sorted(set(ids))
    for fname in author_id_file_candidates:
        p = Path(fname)
        if p.exists():
            try:
                raw = p.read_text(encoding="utf-8")
            except Exception:
                raw = p.read_text(errors="ignore")
            for line in raw.splitlines():
                t = line.split("#", 1)[0].strip()
                if t.isdigit():
                    ids.append(t)
            if ids:
                return sorted(set(ids))
    return []

# ----------------------- CiteScore path --------------------------------

def _candidate_files_in_dir(d: Path) -> List[Path]:
    return [p for name in POSSIBLE_CS_FILENAMES if (p := d / name).exists() and p.is_file()]

def resolve_citescore_path(arg: Optional[str], no_prompt: bool = False) -> Optional[Path]:
    if arg:
        p = Path(arg)
        if p.exists():
            if p.is_file(): return p
            if p.is_dir():
                cand = _candidate_files_in_dir(p)
                if cand: return cand[0]
    envp = os.environ.get("CITESCORE_CSV")
    if envp:
        p = Path(envp)
        if p.exists() and p.is_file(): return p
    for d in DEFAULT_USER_CITESCORE_DIRS + DEFAULT_CITESCORE_DIRS:
        if d.exists() and d.is_dir():
            cand = _candidate_files_in_dir(d)
            if cand: return cand[0]
    if not no_prompt:
        try:
            path_in = input("Path to CiteScore CSV/XLSX (or folder containing it): ").strip('"').strip()
            if path_in:
                p = Path(path_in)
                if p.exists():
                    if p.is_file(): return p
                    if p.is_dir():
                        cand = _candidate_files_in_dir(p)
                        if cand: return cand[0]
        except EOFError:
            pass
    return None

# ----------------------- Read CiteScore table --------------------------

def robust_read_table(path: Path) -> pd.DataFrame:
    if not path.exists():
        raise FileNotFoundError(f"CiteScore file not found: {path}")
    suffix = path.suffix.lower()
    if suffix in (".xlsx", ".xls"):
        return pd.read_excel(path)
    encodings = ["utf-8-sig", "utf-16", "utf-16le", "utf-16be", "cp1254", "iso-8859-9", "cp1252", "latin1"]
    last_err = None
    for enc in encodings:
        try:
            return pd.read_csv(path, encoding=enc, engine="python", sep=None)
        except Exception as e:
            last_err = e
            continue
    try:
        return pd.read_csv(path, encoding="latin1", engine="python", sep=None, on_bad_lines="skip")
    except TypeError:
        return pd.read_csv(path, encoding="latin1", engine="python", sep=None, error_bad_lines=False)  # type: ignore

def load_citescore_table(path: Path) -> pd.DataFrame:
    """
    Return columns:
      source_id (if present), issn_key, eissn_key, asjc_set (frozenset), cs_percentile, citescore
    Accepts flexible column names (case-insensitive)
    """
    cs = robust_read_table(path)
    norm = {c.strip().lower(): c for c in cs.columns}

    p = norm.get("print issn") or norm.get("p-issn") or norm.get("issn")
    e = norm.get("e-issn") or norm.get("eissn")
    pct = norm.get("percentile") or norm.get("citescore percentile")
    val = norm.get("citescore") or norm.get("citescore 2024")
    src = norm.get("source id") or norm.get("scopus source id") or norm.get("scopus sourceid")
    asjc_col = norm.get("asjc") or norm.get("asjc code") or norm.get("asjc codes") or norm.get("subject area asjc")

    if not all([p, e, pct, val]):
        raise KeyError(
            f"CiteScore table must include 'Print ISSN', 'E-ISSN', 'Percentile', 'CiteScore'. Found: {list(cs.columns)}"
        )

    # Standardize names
    cs = cs.rename(columns={p: "print_issn", e: "e_issn", pct: "cs_percentile", val: "citescore"})
    if src:
        cs = cs.rename(columns={src: "source_id"})
    if asjc_col:
        cs = cs.rename(columns={asjc_col: "asjc_raw"})
    else:
        cs["asjc_raw"] = ""

    # Coerce types
    cs["print_issn"] = cs["print_issn"].astype(str)
    cs["e_issn"] = cs["e_issn"].astype(str)
    cs["cs_percentile"] = cs["cs_percentile"].apply(_coerce_percentile)
    cs["citescore"] = cs["citescore"].apply(_coerce_float)
    if "source_id" in cs.columns:
        cs["source_id"] = cs["source_id"].astype(str).str.extract(r"(\d+)", expand=False).fillna("")

    # Normalized keys
    cs["issn_key"] = cs["print_issn"].map(_norm_issn)
    cs["eissn_key"] = cs["e_issn"].map(_norm_issn)

    # ASJC as frozenset (hashable)
    cs["asjc_set"] = cs["asjc_raw"].apply(lambda v: frozenset(_norm_asjc_codes(v)))

    # Keep minimal columns
    keep = ["issn_key", "eissn_key", "asjc_set", "cs_percentile", "citescore"]
    if "source_id" in cs.columns:
        keep.insert(0, "source_id")
    cs = cs[keep]
    return cs

# ----------------------- Scopus helpers --------------------------------

def _extract_issns(ar: Any) -> Tuple[str, str]:
    p = ""; e = ""
    if hasattr(ar, "eIssn"):
        e = _s(getattr(ar, "eIssn"))
    if not e and hasattr(ar, "e_issn"):
        e = _s(getattr(ar, "e_issn"))
    if hasattr(ar, "issn"):
        obj = getattr(ar, "issn")
        if isinstance(obj, str):
            if "ISSN(" in obj:
                mp = re.search(r"print\s*=\s*'([^']+)'", obj)
                me = re.search(r"electronic\s*=\s*'([^']+)'", obj)
                if mp: p = mp.group(1)
                if me and not e: e = me.group(1)
            else:
                p = _s(obj)
        else:
            try:
                p_obj = getattr(obj, "print", ""); e_obj = getattr(obj, "electronic", "")
                if p_obj: p = _s(p_obj)
                if not e and e_obj: e = _s(e_obj)
            except Exception:
                txt = _s(obj)
                mp = re.search(r"print\s*=\s*'([^']+)'", txt)
                me = re.search(r"electronic\s*=\s*'([^']+)'", txt)
                if mp: p = mp.group(1)
                if me and not e: e = me.group(1)
    return p, e


def _extract_asjc(ar: Any) -> Tuple[str, str, str, Set[str]]:
    codes: Set[str] = set(); areas: Set[str] = set(); abbrevs: Set[str] = set()
    sa = getattr(ar, "subject_areas", None)
    if sa:
        try:
            for it in sa:
                c = getattr(it, "code", None)
                a = getattr(it, "area", None)
                ab = getattr(it, "abbrev", None)
                if c is not None:
                    codes.add(f"{int(c):04d}" if str(c).isdigit() else str(c))
                if a: areas.add(_s(a))
                if ab: abbrevs.add(_s(ab))
        except Exception:
            try:  # dict-like
                for it in sa:
                    c = it.get("code"); a = it.get("area"); ab = it.get("abbrev")
                    if c is not None:
                        codes.add(f"{int(c):04d}" if str(c).isdigit() else str(c))
                    if a: areas.add(_s(a))
                    if ab: abbrevs.add(_s(ab))
            except Exception:
                pass
    return ", ".join(sorted(codes)), ", ".join(sorted(areas)), ", ".join(sorted(abbrevs)), codes

# ----------------------- ISSN → Source ID (optional) -------------------

def fetch_source_id_for_issn(issn: str) -> Optional[str]:
    issn = _norm_issn(issn)
    if not issn:
        return None
    try:
        res = SerialTitle(issn)
        items: Iterable[Any]
        try:
            items = list(res) if isinstance(res, (list, tuple)) else [res]
        except Exception:
            items = [res]
        for it in items:
            for attr in ("source_id", "sourcerecord_id", "sourceid"):
                if hasattr(it, attr):
                    sid = _s(getattr(it, attr))
                    if sid.isdigit():
                        return sid
        return None
    except Exception as e:
        warnings.warn(f"SerialTitle lookup failed for ISSN {issn}: {e}")
        return None


def build_cs_by_source(cs_table: pd.DataFrame, serial_sleep: float = 0.1) -> pd.DataFrame:
    """Ensure a cs_by_source table with 'source_id' present, mapping from cs_table.
    If cs_table already has source_id, just dedupe and return.
    Otherwise, look up source_id from ISSN/e-ISSN via SerialTitle.
    """
    if "source_id" in cs_table.columns:
        df = cs_table.copy()
    else:
        df = cs_table.copy()
        # NOTE: These are vectorized calls; add throttling inside fetch if needed.
        df["source_id_from_print"] = df["issn_key"].apply(fetch_source_id_for_issn)
        if serial_sleep:
            time.sleep(serial_sleep)
        df["source_id_from_e"] = df["eissn_key"].apply(fetch_source_id_for_issn)
        df["source_id"] = df["source_id_from_print"].where(df["source_id_from_print"].notna(), df["source_id_from_e"])
        df = df.drop(columns=["source_id_from_print", "source_id_from_e"], errors="ignore")
    # Keep only rows with a source_id
    df = df[df["source_id"].astype(str).str.strip().ne("")].copy()
    # Deduplicate by (source_id, asjc_set); asjc_set is frozenset → hashable
    df = df.drop_duplicates(subset=["source_id", "asjc_set"], keep="first")
    return df[["source_id", "asjc_set", "cs_percentile", "citescore"]]

# ----------------------- Matching logic --------------------------------

def _pick_best_candidate(cands: pd.DataFrame, article_asjc: Set[str]) -> Tuple[Optional[float], Optional[float]]:
    """Choose best row: prefer ASJC overlap, then highest percentile, else first.
    Returns (cs_percentile, citescore).
    """
    if cands is None or cands.empty:
        return None, None
    # compute overlap count
    over = []
    for _i, row in cands.iterrows():
        cs_set = row.get("asjc_set") or set()
        # Accept frozenset/list/tuple/set/string
        if isinstance(cs_set, (set, frozenset)):
            cs_set2 = set(cs_set)
        elif isinstance(cs_set, (list, tuple)):
            cs_set2 = {str(x) for x in cs_set}
        else:
            cs_set2 = _norm_asjc_codes(cs_set)
        over.append(len(article_asjc & cs_set2))
    cands = cands.copy()
    cands["_overlap"] = over
    # sort by: overlap desc, percentile desc (None last)
    cands["_pct"] = cands["cs_percentile"].fillna(-1e9)
    cands = cands.sort_values(["_overlap", "_pct"], ascending=[False, False])
    top = cands.iloc[0]
    return top.get("cs_percentile"), top.get("citescore")


def enrich_with_citescore_sourceid_asjc(
    df_articles: pd.DataFrame,
    cs_table: pd.DataFrame,
    cs_by_source: pd.DataFrame,
) -> pd.DataFrame:
    if df_articles is None or df_articles.empty:
        return df_articles.copy()

    df = df_articles.copy()
    for col in ("issn_print", "issn_electronic", "source_id"):
        if col not in df.columns:
            df[col] = ""

    # Prepare article keys
    df["issn_key"] = df["issn_print"].astype(str).map(_norm_issn)
    df["eissn_key"] = df["issn_electronic"].astype(str).map(_norm_issn)

    # Pre-split article ASJC sets
    a_asjc_sets: List[Set[str]] = []
    for v in df.get("asjc_codes", pd.Series([""] * len(df))):
        a_asjc_sets.append(_norm_asjc_codes(v))

    cs_p = cs_table[["issn_key", "asjc_set", "cs_percentile", "citescore"]].drop_duplicates()
    cs_e = cs_table[["eissn_key", "asjc_set", "cs_percentile", "citescore"]].drop_duplicates()

    out_pct: List[Optional[float]] = []
    out_val: List[Optional[float]] = []

    for idx, row in df.iterrows():
        article_asjc = a_asjc_sets[idx] if idx < len(a_asjc_sets) else set()
        sid = _s(row.get("source_id"))
        pct = None; val = None
        # 1) try source_id
        if sid:
            cands = cs_by_source[cs_by_source["source_id"].astype(str) == sid]
            pct, val = _pick_best_candidate(cands, article_asjc)
        # 2) fallback by ISSN
        if pct is None and val is None:
            issn = _s(row.get("issn_key"))
            eissn = _s(row.get("eissn_key"))
            cands = pd.concat([
                cs_p[cs_p["issn_key"] == issn],
                cs_e[cs_e["eissn_key"] == eissn],
            ], ignore_index=True)
            pct, val = _pick_best_candidate(cands, article_asjc)
        out_pct.append(pct)
        out_val.append(val)

    df["cs_percentile"] = out_pct
    df["citescore"] = out_val
    df["quartile"] = df["cs_percentile"].apply(quartile_from_percentile)
    return df

# ----------------------- Scopus collectors (with per-author aff detection) -------

def get_author_name(author_id: str) -> str:
    try:
        ar = AuthorRetrieval(author_id)
        name = f"{_s(ar.given_name)} {_s(ar.surname)}".strip()
        return name or author_id
    except Exception:
        return author_id

def get_author_eids(author_id: str) -> List[str]:
    """
    Return EIDs for the given author. We fetch by AU-ID only (no AF-ID here)
    and then perform per-article per-author affiliation detection so we can
    precisely determine whether the *author on that paper* was affiliated with
    AFF_ID on that particular record.
    """
    try:
        s = ScopusSearch(f"AU-ID({author_id})", subscriber=True)
        return s.get_eids() or []
    except Exception as e:
        warnings.warn(f"ScopusSearch failed for AU-ID({author_id}): {e}")
        return []

def _safe_int(v) -> Optional[int]:
    try:
        return int(v)
    except Exception:
        return None

def get_article_metadata(eid: str, target_auid: Optional[str] = None):
    """
    Return article metadata dict. Also detect whether the given target_auid
    appears in the article and whether that author entry has affiliation_id == AFF_ID.
    This replicates the per-author affiliation isolation logic used in the longer ETL script.
    """
    try:
        ar = AbstractRetrieval(eid, view="FULL")
    except ScopusException as e:
        warnings.warn(f"AbstractRetrieval failed for {eid}: {e}")
        return None

    # keep only articles and reviews for general collection; APP will only count subtype == 'ar'
    if ar.subtype not in ("ar", "re"):
        return None

    year = ""
    if getattr(ar, "coverDate", None):
        year = _s(ar.coverDate)[:4]

    issn_print, issn_elec = _extract_issns(ar)
    asjc_codes_csv, asjc_areas_csv, asjc_abbrevs_csv, _codes_set = _extract_asjc(ar)

    # Build author-affiliation structures from authorgroup where available
    bau_flag = False  # whether target_auid is present with AFF_ID on this paper
    bau_orgs = []     # organizations for bau-affiliated author(s)
    auid_org_map: Dict[str, str] = {}
    auth_err = None
    groups = getattr(ar, "authorgroup", None)
    if groups:
        try:
            for g in groups:
                try:
                    g_auid = str(getattr(g, "auid", "") or "")
                    org_val = getattr(g, "organization", None)
                    aff_id_val = _safe_int(getattr(g, "affiliation_id", None))
                    if g_auid:
                        auid_org_map[g_auid] = _s(org_val)
                    # if this group entry belongs to our target author and affiliation id matches AFF_ID
                    if target_auid and g_auid == str(target_auid) and AFF_ID and aff_id_val == _safe_int(AFF_ID):
                        bau_flag = True
                        if org_val:
                            bau_orgs.append(_s(org_val))
                except Exception:
                    continue
        except Exception as ex:
            # fallback: try authors list mapping if authorgroup parsing failed
            auth_err = str(ex)
    else:
        # authorgroup not present — try to use 'authors' entries and affiliations
        try:
            authors = getattr(ar, "authors", None) or []
            for a in authors:
                try:
                    g_auid = str(getattr(a, "auid", "") or "")
                    # some authors may have affiliation info inline
                    org_val = getattr(a, "orgname", None) or getattr(a, "affiliation", None) or ""
                    if g_auid:
                        auid_org_map[g_auid] = _s(org_val)
                    # affiliation id may not be present here — can't reliably detect AFF_ID
                except Exception:
                    continue
        except Exception:
            pass

    # Additional: collect affiliation list for the record (for collaboration detection)
    insts = getattr(ar, "affiliation", None) or []

    return {
        "eid": _s(eid),
        "title": _s(ar.title),
        "year": _s(year),
        "publication_name": _s(getattr(ar, "publicationName", "")),
        "subtype": _s(ar.subtype),
        "doi": _s(getattr(ar, "doi", "")),
        "source_id": _s(getattr(ar, "source_id", "")),
        "issn_print": _s(issn_print),
        "issn_electronic": _s(issn_elec),
        "asjc_codes": asjc_codes_csv,
        "asjc_areas": asjc_areas_csv,
        "asjc_abbrevs": asjc_abbrevs_csv,
        "authors_count": len(ar.authors) if getattr(ar, "authors", None) else 1,
        "combined": "; ".join([t for t in (getattr(ar, "authkeywords", []) or []) if _s(t)]),
        "abstract": _s(getattr(ar, "description", "")),
        # affiliation-detection fields:
        "author_org_map_json": json.dumps(auid_org_map, ensure_ascii=False),
        "is_bau_author": bau_flag,
        "bau_author_orgs": "; ".join(dict.fromkeys(bau_orgs)),
        "affiliations_list": [(getattr(i, "name", "") if hasattr(i, "name") else _s(i)) for i in insts],
        "auth_group_error": auth_err,
    }

# ----------------------- APP calculation helpers -----------------------

def _qc_from_percentile(p: Optional[float]) -> Optional[float]:
    """QC mapping assuming higher percentile is better (>=90 is Top 10%)."""
    if p is None:
        return None
    try:
        p = float(p)
    except Exception:
        return None
    if p >= 90:   # Top 10%
        return 1.4
    if p >= 75:   # Q1
        return 1.0
    if p >= 50:   # Q2
        return 0.8
    if p >= 25:   # Q3
        return 0.6
    if p >= 0:    # Q4
        return 0.4
    return None

def _ac_from_authors(n: Any) -> float:
    """Author Coefficient: 1.2 if single author else 1.2 / n_authors."""
    try:
        n = int(n)
    except Exception:
        n = 1
    return 1.2 if n <= 1 else 1.2 / max(n, 1)

def _to_int_year(y: Any) -> Optional[int]:
    try:
        s = str(y)
        m = re.search(r"\d{4}", s)
        return int(m.group(0)) if m else None
    except Exception:
        return None

def build_app_sheet(df_articles: pd.DataFrame, now_year: Optional[int] = None) -> Tuple[pd.DataFrame, Dict[str, Any]]:
    """
    Returns (df_app, summary) where df_app contains the per-paper APP breakdown and
    summary has 'app_total' and 'eligibility' text.

    NOTE: APP is calculated only for journal articles with subtype == "ar".
    """
    if df_articles is None or df_articles.empty:
        return pd.DataFrame(), {"app_total": 0.0, "eligibility": "No eligible items", "years": []}

    # Determine the 3-year window (includes current year)
    cy = now_year or datetime.now().year
    years_ok = {cy, cy - 1, cy - 2}

    # Coerce needed fields
    tmp = df_articles.copy()
    tmp["year_i"] = tmp.get("year", "").apply(_to_int_year)
    tmp["cs_percentile_num"] = pd.to_numeric(tmp.get("cs_percentile"), errors="coerce")
    tmp["authors_count_i"] = pd.to_numeric(tmp.get("authors_count"), errors="coerce").fillna(1).astype(int)
    tmp["subtype_norm"] = tmp.get("subtype", "").astype(str).str.lower()

    # Eligibility: last 3 calendar years, subtype == "ar" only, has percentile
    eligible = tmp[
        tmp["year_i"].isin(years_ok) &
        tmp["cs_percentile_num"].notna() &
        (tmp["subtype_norm"] == "ar")
    ].copy()

    if eligible.empty:
        return pd.DataFrame(), {"app_total": 0.0, "eligibility": "No eligible items (no 'ar' articles in window)", "years": sorted(years_ok)}

    # Compute QC, AC, Contribution
    eligible["QC"] = eligible["cs_percentile_num"].apply(_qc_from_percentile)
    eligible["AC"] = eligible["authors_count_i"].apply(_ac_from_authors)
    # If QC missing after mapping, treat as ineligible
    eligible = eligible[eligible["QC"].notna()].copy()
    if eligible.empty:
        return pd.DataFrame(), {"app_total": 0.0, "eligibility": "No eligible items (QC missing)", "years": sorted(years_ok)}

    eligible["Contribution"] = eligible["AC"] * eligible["QC"]

    # Round to 2 decimals per policy
    eligible["AC"] = eligible["AC"].round(2)
    eligible["QC"] = eligible["QC"].round(2)
    eligible["Contribution"] = eligible["Contribution"].round(2)

    # Select user-friendly columns
    out_cols = [
        "eid", "title", "year", "publication_name",
        "authors_count", "cs_percentile", "quartile",
        "AC", "QC", "Contribution"
    ]
    for c in out_cols:
        if c not in eligible.columns:
            eligible[c] = pd.Series(dtype="object")
    df_app = eligible[out_cols].sort_values(["year", "Contribution"], ascending=[False, False]).reset_index(drop=True)

    app_total = float(df_app["Contribution"].sum().round(2))

    # Eligibility band (based on your table)
    if app_total > 1.0:
        elig = "APP > 1.0 → up to 2 supports / AY (only 1 requires full indexing & APP check)"
    elif app_total >= 0.4:
        elig = "0.4 ≤ APP ≤ 1.0 → 1 support / AY"
    else:
        elig = "APP < 0.4 → 1 support / AY (if other criteria met)"

    summary = {"app_total": round(app_total, 2), "eligibility": elig, "years": sorted(years_ok)}
    return df_app, summary

# ----------------------- Excel writer (with APP sheet) -----------------

def _write_excel_xlsxwriter(df: pd.DataFrame, path: Path, app_df: Optional[pd.DataFrame] = None, app_summary: Optional[Dict[str, Any]] = None):
    with pd.ExcelWriter(path, engine="xlsxwriter") as xl:
        df.to_excel(xl, sheet_name="articles", index=False)
        ws = xl.sheets["articles"]
        try:
            ws.freeze_panes(1, 0)
            ws.set_column("A:A", 20)  # eid
            ws.set_column("B:B", 50)  # title
            ws.set_column("C:C", 8)   # year
            ws.set_column("D:D", 36)  # publication_name
            ws.set_column("E:E", 8)   # subtype
            ws.set_column("F:F", 26)  # doi
            ws.set_column("G:G", 14)  # source_id
            ws.set_column("H:I", 14)  # ISSNs
            ws.set_column("J:J", 18)  # asjc_codes
            ws.set_column("K:K", 26)  # asjc_areas
            ws.set_column("L:L", 14)  # asjc_abbrevs
            ws.set_column("M:N", 14)  # cs_percentile / citescore
            ws.set_column("O:O", 10)  # quartile
            ws.set_column("P:P", 12)  # authors_count
            ws.set_column("Q:Q", 28)  # combined
            ws.set_column("R:R", 80)  # abstract
            # extra affiliation columns
            ws.set_column("S:S", 12)  # is_bau_author
            ws.set_column("T:T", 28)  # bau_author_orgs
            ws.set_column("U:U", 36)  # author_org_map_json
        except Exception:
            pass

        # Sheet 2: APP (optional)
        if app_df is not None and not app_df.empty:
            # write header and summary first (we'll write table starting at row 6)
            ws2 = xl.book.add_worksheet("APP")
            xl.sheets["APP"] = ws2  # ensure mapping exists
            try:
                ws2.write(0, 0, "APP calculation — last 3 calendar years (journal articles only; subtype == 'ar')")
                if app_summary:
                    ws2.write(1, 0, "Years considered")
                    ws2.write(1, 1, ", ".join(str(y) for y in app_summary.get("years", [])))
                    ws2.write(2, 0, "APP Score")
                    ws2.write(2, 1, app_summary.get("app_total", 0.0))
                    ws2.write(3, 0, "Eligibility")
                    ws2.write(3, 1, app_summary.get("eligibility", ""))
                # write the dataframe starting at row 6 (index 5) to leave space for header
                (app_df.reset_index(drop=True)).to_excel(xl, sheet_name="APP", index=False, startrow=5)
                # adjust columns
                ws2.freeze_panes(6, 0)
                ws2.set_column("A:A", 20)  # eid
                ws2.set_column("B:B", 60)  # title
                ws2.set_column("C:C", 8)   # year
                ws2.set_column("D:D", 36)  # journal
                ws2.set_column("E:E", 12)  # authors_count
                ws2.set_column("F:F", 14)  # cs_percentile
                ws2.set_column("G:G", 10)  # quartile
                ws2.set_column("H:J", 14)  # AC, QC, Contribution
            except Exception:
                # Fallback: write simply by pandas
                try:
                    app_df.to_excel(xl, sheet_name="APP", index=False)
                except Exception:
                    pass

def _write_excel_openpyxl(df: pd.DataFrame, path: Path, app_df: Optional[pd.DataFrame] = None, app_summary: Optional[Dict[str, Any]] = None):
    with pd.ExcelWriter(path, engine="openpyxl") as xl:
        df.to_excel(xl, sheet_name="articles", index=False)
        try:
            ws = xl.sheets["articles"]
            ws.freeze_panes = "A2"
        except Exception:
            pass
        if app_df is not None and not app_df.empty:
            try:
                app_df.to_excel(xl, sheet_name="APP", index=False, startrow=5)
                ws2 = xl.sheets.get("APP", None)
                if ws2 is not None:
                    ws2.freeze_panes = "A6"
            except Exception:
                try:
                    app_df.to_excel(xl, sheet_name="APP", index=False)
                except Exception:
                    pass

def save_author_excel(author_id: str, author_name: str, df_articles: pd.DataFrame, out_dir: Path) -> str:
    out_dir.mkdir(parents=True, exist_ok=True)
    path = out_dir / make_author_filename(author_name, author_id)
    need = [
        "eid","title","year","publication_name","subtype","doi",
        "source_id","issn_print","issn_electronic",
        "asjc_codes","asjc_areas","asjc_abbrevs",
        "cs_percentile","citescore","quartile",
        "authors_count","combined","abstract",
        # affiliation-specific
        "is_bau_author","bau_author_orgs","author_org_map_json"
    ]
    df = df_articles.copy()
    for c in need:
        if c not in df.columns:
            df[c] = pd.Series(dtype="object")
    df = df[need]

    # Build APP breakdown & summary (APP counts only subtype == 'ar')
    app_df, app_summary = build_app_sheet(df)

    try:
        _write_excel_xlsxwriter(df, path, app_df=app_df if not app_df.empty else None, app_summary=app_summary)
    except Exception:
        _write_excel_openpyxl(df, path, app_df=app_df if not app_df.empty else None, app_summary=app_summary)
    return str(path)

# ----------------------- Orchestration --------------------------------

def process_author(author_id: str, cs_table: pd.DataFrame, out_dir: Path, sleep: float, serial_sleep: float, cs_by_source: Optional[pd.DataFrame] = None) -> Optional[str]:
    try:
        name = get_author_name(author_id)
        print(f"→ AU-ID {author_id} — {name} (AFF_ID filter: {AFF_ID})")
        eids = get_author_eids(author_id)
        recs: List[Dict[str, Any]] = []
        for eid in eids:
            md = get_article_metadata(eid, target_auid=author_id)
            # If AFF_ID specified, only keep records where this author is affiliated with AFF_ID on the paper
            if md:
                if AFF_ID:
                    if not md.get("is_bau_author", False):
                        # skip this article — the author is not shown as affiliated with AFF_ID on this record
                        if sleep:
                            time.sleep(sleep)
                        continue
                # attach Scopus-derived fields remapped to column names expected downstream
                rec = {
                    "eid": md.get("eid"),
                    "title": md.get("title"),
                    "year": md.get("year"),
                    "publication_name": md.get("publication_name"),
                    "subtype": md.get("subtype"),
                    "doi": md.get("doi"),
                    "source_id": md.get("source_id"),
                    "issn_print": md.get("issn_print"),
                    "issn_electronic": md.get("issn_electronic"),
                    "asjc_codes": md.get("asjc_codes"),
                    "asjc_areas": md.get("asjc_areas"),
                    "asjc_abbrevs": md.get("asjc_abbrevs"),
                    "authors_count": md.get("authors_count"),
                    "combined": md.get("combined"),
                    "abstract": md.get("abstract"),
                    # affiliation detection fields (new)
                    "is_bau_author": md.get("is_bau_author"),
                    "bau_author_orgs": md.get("bau_author_orgs"),
                    "author_org_map_json": md.get("author_org_map_json"),
                }
                recs.append(rec)
            if sleep:
                time.sleep(sleep)
        df = pd.DataFrame(recs)

        if not df.empty:
            # build cs_by_source if not provided (single lookup per run is preferred)
            if cs_by_source is None:
                cs_by_source = build_cs_by_source(cs_table, serial_sleep=serial_sleep)
            df = enrich_with_citescore_sourceid_asjc(df, cs_table, cs_by_source)

        path = save_author_excel(author_id, name, df if not df.empty else pd.DataFrame(), out_dir)
        print(f"   ✓ Wrote {Path(path).name}  ({0 if df is None else len(df)} rows)")
        return path
    except Exception as e:
        print(f"   ⚠️ Failed {author_id}: {e}")
        return None

# ----------------------- Public API (Jupyter) -------------------------

def run(
    auids: Optional[str] = None,
    citescore: Optional[str] = None,
    outdir: str = "authors",
    sleep: float = 0.05,
    serial_sleep: float = 0.1,
    no_prompt: bool = True,
    aff_id: Optional[str] = None,
) -> None:
    """
    Jupyter-friendly entrypoint. Pass aff_id to override the default AFF_ID filter.
    """
    global AFF_ID
    if aff_id is not None:
        AFF_ID = aff_id

    out_dir = Path(outdir)
    cs_path = resolve_citescore_path(citescore, no_prompt=no_prompt)
    if not cs_path:
        raise FileNotFoundError(
            "Could not locate CiteScore file. Pass citescore=..., set CITESCORE_CSV env var, or place the file in a default folder."
        )
    print(f"Using CiteScore file: {cs_path}")
    cs_table = load_citescore_table(cs_path)

    ids = read_auids_from_cli_or_file(auids)
    if not ids:
        print("No AU-IDs provided. Provide auids='555...,572...' or authors.txt file.")
        return
    print(f"CiteScore rows: {len(cs_table)} | Authors: {len(ids)} | Affiliation filter: {AFF_ID}")

    # Build cs_by_source once per run to avoid repeated SerialTitle lookups
    cs_by_source = build_cs_by_source(cs_table, serial_sleep=serial_sleep)

    out_dir.mkdir(parents=True, exist_ok=True)
    for i, auid in enumerate(ids, 1):
        print(f"[{i}/{len(ids)}]")
        process_author(auid, cs_table, out_dir, sleep=sleep, serial_sleep=serial_sleep, cs_by_source=cs_by_source)
    print(f"\nDone. Files saved to: {out_dir.resolve()}")

# ----------------------- CLI -----------------------------------------

def main(argv: Optional[list] = None):
    global AFF_ID
    ap = argparse.ArgumentParser(description="Fetch Scopus pubs by AU-ID and match CiteScore by Source ID then ASJC (ISSN fallback). Per-article affiliation detection isolates items where the author is affiliated with the configured aff_id on that paper.")
    ap.add_argument("--auids", type=str, default=None, help="Comma/space-separated Scopus Author IDs. If omitted, reads authors.txt")
    ap.add_argument("--citescore", type=str, default=None, help="Path to CiteScore CSV/XLSX, or a folder containing it")
    ap.add_argument("--outdir", type=str, default="authors", help="Output directory (default: ./authors)")
    ap.add_argument("--sleep", type=float, default=0.05, help="Sleep between EID fetches (seconds)")
    ap.add_argument("--serial-sleep", type=float, default=0.1, help="Sleep between SerialTitle ISSN lookups (seconds)")
    ap.add_argument("--no-prompt", action="store_true", help="Do not prompt for missing CiteScore path; exit with error")
    ap.add_argument("--aff-id", type=str, default=None, help=f"Optional affiliation ID to isolate author-affiliated publications (default: {AFF_ID_DEFAULT})")
    args, _unknown = ap.parse_known_args(argv)

    if args.aff_id is not None:
        AFF_ID = args.aff_id

    cs_path = resolve_citescore_path(args.citescore, no_prompt=args.no_prompt)
    if not cs_path:
        if _is_ipython() and not args.no_prompt:
            try:
                entered = input("Enter CiteScore CSV/XLSX path or containing folder: ").strip('"').strip()
            except EOFError:
                entered = ""
            if entered:
                cs_path = resolve_citescore_path(entered, no_prompt=True)
        if not cs_path:
            print("❌ Could not locate the CiteScore file. Pass --citescore or set CITESCORE_CSV.")
            if _is_ipython():
                return
            sys.exit(2)
    print(f"Using CiteScore file: {cs_path}")
    cs_table = load_citescore_table(cs_path)

    auids = read_auids_from_cli_or_file(args.auids)
    if not auids and _is_ipython():
        try:
            entered = input("Enter Scopus Author IDs (comma or space separated), or leave blank to cancel: ").strip()
        except EOFError:
            entered = ""
        if entered:
            auids = read_auids_from_cli_or_file(entered)
    if not auids:
        print("No AU-IDs provided. Use --auids, or create authors.txt with one AU-ID per line.")
        if _is_ipython():
            return
        sys.exit(0)

    out_dir = Path(args.outdir)
    print(f"CiteScore rows: {len(cs_table)} | Authors: {len(auids)} | Out: {out_dir.resolve()} | Affiliation filter: {AFF_ID}")
    out_dir.mkdir(parents=True, exist_ok=True)

    # Build cs_by_source once for the whole run
    cs_by_source = build_cs_by_source(cs_table, serial_sleep=args.serial_sleep)

    written = []
    for i, auid in enumerate(auids, 1):
        print(f"[{i}/{len(auids)}] Processing AU-ID {auid} … (AFF_ID={AFF_ID})")
        p = process_author(auid, cs_table, out_dir, sleep=args.sleep, serial_sleep=args.serial_sleep, cs_by_source=cs_by_source)
        if p:
            written.append(p)

    print(f"\nDone. Wrote {len(written)} file(s) to {out_dir.resolve()}.")

if __name__ == "__main__":
    main()


Using CiteScore file: C:\Users\yusef.atteyih\Desktop\Academic Research Unit\Yusef ATTEYIH\Data Solutions\Data Solutions 2.0\APP Calculation\CiteScore 2024\CiteScore 2024 annual values.csv


Enter Scopus Author IDs (comma or space separated), or leave blank to cancel:  55042518500


CiteScore rows: 75679 | Authors: 1 | Out: C:\Users\yusef.atteyih\Desktop\Academic Research Unit\Yusef ATTEYIH\Data Solutions\Data Solutions 2.0\APP Calculation\authors | Affiliation filter: 60021379
[1/1] Processing AU-ID 55042518500 … (AFF_ID=60021379)
→ AU-ID 55042518500 — Turker D. Kilic (AFF_ID filter: 60021379)
   ✓ Wrote _turker_d_kilic___55042518500.xlsx  (72 rows)

Done. Wrote 1 file(s) to C:\Users\yusef.atteyih\Desktop\Academic Research Unit\Yusef ATTEYIH\Data Solutions\Data Solutions 2.0\APP Calculation\authors.


In [36]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
r"""
Scopus → CiteScore with disambiguation by **Source ID then ASJC (sub-subject area)**

This version:
• Stores ASJC sets as frozenset (hashable).
• Matches publications to CiteScore by Source ID then ASJC (ISSN fallback).
• Performs per-article author→affiliation mapping and isolates items where the
  author is affiliated with the configured AFF_ID on that paper.
• Produces three sheets in each author workbook:
    - "Articles"      : ALL articles for the author (no affiliation filtering)
    - "BAU Articles"  : only those articles where the author is affiliated with AFF_ID
    - "APP"           : Participation Score computed from BAU Articles (subtype == "ar")
• APP uses Author Coefficient (AC = 1.2 if sole author else 1.2 / n_authors)
  and Quartile Coefficient (QC) mapped from CiteScore percentile,
  Contribution = AC × QC, summed and rounded.

Usage:
  - Jupyter: run(auids="12345,67890", citescore="path/to/CiteScore.csv", aff_id="60021379")
  - CLI: python this_script.py --auids 57193254610 --citescore "CiteScore 2024.csv" --aff-id 60021379

Requirements:
  pybliometrics, pandas, xlsxwriter or openpyxl
  Configure pybliometrics Scopus API credentials
"""

from __future__ import annotations

import argparse
import json
import os
import re
import sys
import time
import unicodedata
import warnings
from datetime import datetime
from pathlib import Path
from typing import Any, Dict, Iterable, List, Optional, Set, Tuple

import pandas as pd

# --- Scopus (pybliometrics) ---
try:
    from pybliometrics.scopus import AbstractRetrieval, AuthorRetrieval, ScopusSearch, SerialTitle
    from pybliometrics.scopus.exception import ScopusException
except Exception as e:
    raise RuntimeError(
        "pybliometrics is required. Install with `pip install pybliometrics` "
        "and ensure your Scopus API credentials are configured."
    ) from e

# ----------------------- IPython detection ----------------------------

def _is_ipython() -> bool:
    return ("ipykernel" in sys.modules) or ("IPython" in sys.modules)

# ----------------------- Defaults & Affiliation filter ----------------

AFF_ID_DEFAULT = "60021379"
AFF_ID: Optional[str] = AFF_ID_DEFAULT

DEFAULT_USER_CITESCORE_DIRS = [
    Path(r"C:\\Users\\yusef.atteyih\\Desktop\\Academic Research Unit\\Yusef ATTEYIH\\Data Solutions\\Data Solutions 2.0\\APP Calculation\\CiteScore 2024"),
]
DEFAULT_CITESCORE_DIRS = [Path("CiteScore 2024"), Path(".")]
POSSIBLE_CS_FILENAMES = [
    "CiteScore 2024 annual values.csv",
    "CiteScore 2024 annual values.xlsx",
    "CiteScore 2024.csv",
    "citescore.csv",
    "citescore.xlsx",
]

# ----------------------- Small helpers --------------------------------

def _s(x) -> str:
    try:
        if x is None or (isinstance(x, float) and pd.isna(x)):
            return ""
    except Exception:
        pass
    return x if isinstance(x, str) else str(x)

def _norm_issn(s: str) -> str:
    """Uppercase; keep 0-9 and 'X'; strip others."""
    return re.sub(r"[^0-9X]", "", _s(s).upper())

def _norm_asjc_codes(raw: Any) -> Set[str]:
    """Return a set of 4-digit ASJC codes as strings."""
    out: Set[str] = set()
    if raw is None:
        return out
    if isinstance(raw, (list, tuple, set)):
        it = raw
    else:
        it = re.split(r"[^0-9]", _s(raw))
    for tok in it:
        if tok and tok.isdigit():
            if len(tok) == 4:
                out.add(tok)
            elif len(tok) > 4:
                out.add(tok[-4:])
    return out

def _coerce_percentile(val) -> Optional[float]:
    txt = _s(val).strip()
    if not txt:
        return None
    txt = txt.replace("%", "").replace(" ", "").replace(",", ".")
    m = re.search(r"[-+]?\d*\.?\d+", txt)
    if not m:
        return None
    try:
        return float(m.group(0))
    except Exception:
        return None

def _coerce_float(val) -> Optional[float]:
    txt = _s(val).strip().replace(",", ".")
    if not txt:
        return None
    m = re.search(r"[-+]?\d*\.?\d+", txt)
    if not m:
        return None
    try:
        return float(m.group(0))
    except Exception:
        return None

def quartile_from_percentile(p: Optional[float]) -> str:
    if p is None:
        return ""
    try:
        p = float(p)
    except Exception:
        return ""
    if p >= 90: return "QT"
    if p >= 75: return "Q1"
    if p >= 50: return "Q2"
    if p >= 25: return "Q3"
    return "Q4"

# ----------------------- Filenames ------------------------------------

def _ascii_slug(s: str) -> str:
    s = unicodedata.normalize("NFKD", _s(s)).encode("ascii", "ignore").decode("ascii")
    s = s.lower()
    s = re.sub(r"[^0-9a-z]+", "_", s)
    s = re.sub(r"_+", "_", s).strip("_")
    return s

def make_author_filename(author_name: str, author_id: str) -> str:
    name_slug = _ascii_slug(author_name)
    leading = "_" + name_slug if not name_slug.startswith("_") else name_slug
    return f"{leading}___{author_id}.xlsx"

# ----------------------- Read AU-IDs ----------------------------------

author_id_file_candidates = ("authors.txt", "auids.txt", "ScopusAuthorIDs.txt")

def read_auids_from_cli_or_file(cli_auids: Optional[str]) -> List[str]:
    ids: List[str] = []
    if cli_auids:
        for tok in re.split(r"[,\s]+", cli_auids.strip()):
            if tok and tok.isdigit():
                ids.append(tok)
    if ids:
        return sorted(set(ids))
    for fname in author_id_file_candidates:
        p = Path(fname)
        if p.exists():
            try:
                raw = p.read_text(encoding="utf-8")
            except Exception:
                raw = p.read_text(errors="ignore")
            for line in raw.splitlines():
                t = line.split("#", 1)[0].strip()
                if t.isdigit():
                    ids.append(t)
            if ids:
                return sorted(set(ids))
    return []

# ----------------------- CiteScore path --------------------------------

def _candidate_files_in_dir(d: Path) -> List[Path]:
    return [p for name in POSSIBLE_CS_FILENAMES if (p := d / name).exists() and p.is_file()]

def resolve_citescore_path(arg: Optional[str], no_prompt: bool = False) -> Optional[Path]:
    if arg:
        p = Path(arg)
        if p.exists():
            if p.is_file(): return p
            if p.is_dir():
                cand = _candidate_files_in_dir(p)
                if cand: return cand[0]
    envp = os.environ.get("CITESCORE_CSV")
    if envp:
        p = Path(envp)
        if p.exists() and p.is_file(): return p
    for d in DEFAULT_USER_CITESCORE_DIRS + DEFAULT_CITESCORE_DIRS:
        if d.exists() and d.is_dir():
            cand = _candidate_files_in_dir(d)
            if cand: return cand[0]
    if not no_prompt:
        try:
            path_in = input("Path to CiteScore CSV/XLSX (or folder containing it): ").strip('"').strip()
            if path_in:
                p = Path(path_in)
                if p.exists():
                    if p.is_file(): return p
                    if p.is_dir():
                        cand = _candidate_files_in_dir(p)
                        if cand: return cand[0]
        except EOFError:
            pass
    return None

# ----------------------- Read CiteScore table --------------------------

def robust_read_table(path: Path) -> pd.DataFrame:
    if not path.exists():
        raise FileNotFoundError(f"CiteScore file not found: {path}")
    suffix = path.suffix.lower()
    if suffix in (".xlsx", ".xls"):
        return pd.read_excel(path)
    encodings = ["utf-8-sig", "utf-16", "utf-16le", "utf-16be", "cp1254", "iso-8859-9", "cp1252", "latin1"]
    last_err = None
    for enc in encodings:
        try:
            return pd.read_csv(path, encoding=enc, engine="python", sep=None)
        except Exception as e:
            last_err = e
            continue
    try:
        return pd.read_csv(path, encoding="latin1", engine="python", sep=None, on_bad_lines="skip")
    except TypeError:
        return pd.read_csv(path, encoding="latin1", engine="python", sep=None, error_bad_lines=False)  # type: ignore

def load_citescore_table(path: Path) -> pd.DataFrame:
    """
    Return columns:
      source_id (if present), issn_key, eissn_key, asjc_set (frozenset), cs_percentile, citescore
    """
    cs = robust_read_table(path)
    norm = {c.strip().lower(): c for c in cs.columns}

    p = norm.get("print issn") or norm.get("p-issn") or norm.get("issn")
    e = norm.get("e-issn") or norm.get("eissn")
    pct = norm.get("percentile") or norm.get("citescore percentile")
    val = norm.get("citescore") or norm.get("citescore 2024")
    src = norm.get("source id") or norm.get("scopus source id") or norm.get("scopus sourceid")
    asjc_col = norm.get("asjc") or norm.get("asjc code") or norm.get("asjc codes") or norm.get("subject area asjc")

    if not all([p, e, pct, val]):
        raise KeyError(
            f"CiteScore table must include 'Print ISSN', 'E-ISSN', 'Percentile', 'CiteScore'. Found: {list(cs.columns)}"
        )

    # Standardize names
    cs = cs.rename(columns={p: "print_issn", e: "e_issn", pct: "cs_percentile", val: "citescore"})
    if src:
        cs = cs.rename(columns={src: "source_id"})
    if asjc_col:
        cs = cs.rename(columns={asjc_col: "asjc_raw"})
    else:
        cs["asjc_raw"] = ""

    # Coerce types
    cs["print_issn"] = cs["print_issn"].astype(str)
    cs["e_issn"] = cs["e_issn"].astype(str)
    cs["cs_percentile"] = cs["cs_percentile"].apply(_coerce_percentile)
    cs["citescore"] = cs["citescore"].apply(_coerce_float)
    if "source_id" in cs.columns:
        cs["source_id"] = cs["source_id"].astype(str).str.extract(r"(\d+)", expand=False).fillna("")

    # Normalized keys
    cs["issn_key"] = cs["print_issn"].map(_norm_issn)
    cs["eissn_key"] = cs["e_issn"].map(_norm_issn)

    # ASJC as frozenset (hashable)
    cs["asjc_set"] = cs["asjc_raw"].apply(lambda v: frozenset(_norm_asjc_codes(v)))

    # Keep minimal columns
    keep = ["issn_key", "eissn_key", "asjc_set", "cs_percentile", "citescore"]
    if "source_id" in cs.columns:
        keep.insert(0, "source_id")
    cs = cs[keep]
    return cs

# ----------------------- Scopus helpers --------------------------------

def _extract_issns(ar: Any) -> Tuple[str, str]:
    p = ""; e = ""
    if hasattr(ar, "eIssn"):
        e = _s(getattr(ar, "eIssn"))
    if not e and hasattr(ar, "e_issn"):
        e = _s(getattr(ar, "e_issn"))
    if hasattr(ar, "issn"):
        obj = getattr(ar, "issn")
        if isinstance(obj, str):
            if "ISSN(" in obj:
                mp = re.search(r"print\s*=\s*'([^']+)'", obj)
                me = re.search(r"electronic\s*=\s*'([^']+)'", obj)
                if mp: p = mp.group(1)
                if me and not e: e = me.group(1)
            else:
                p = _s(obj)
        else:
            try:
                p_obj = getattr(obj, "print", ""); e_obj = getattr(obj, "electronic", "")
                if p_obj: p = _s(p_obj)
                if not e and e_obj: e = _s(e_obj)
            except Exception:
                txt = _s(obj)
                mp = re.search(r"print\s*=\s*'([^']+)'", txt)
                me = re.search(r"electronic\s*=\s*'([^']+)'", txt)
                if mp: p = mp.group(1)
                if me and not e: e = me.group(1)
    return p, e


def _extract_asjc(ar: Any) -> Tuple[str, str, str, Set[str]]:
    codes: Set[str] = set(); areas: Set[str] = set(); abbrevs: Set[str] = set()
    sa = getattr(ar, "subject_areas", None)
    if sa:
        try:
            for it in sa:
                c = getattr(it, "code", None)
                a = getattr(it, "area", None)
                ab = getattr(it, "abbrev", None)
                if c is not None:
                    codes.add(f"{int(c):04d}" if str(c).isdigit() else str(c))
                if a: areas.add(_s(a))
                if ab: abbrevs.add(_s(ab))
        except Exception:
            try:  # dict-like
                for it in sa:
                    c = it.get("code"); a = it.get("area"); ab = it.get("abbrev")
                    if c is not None:
                        codes.add(f"{int(c):04d}" if str(c).isdigit() else str(c))
                    if a: areas.add(_s(a))
                    if ab: abbrevs.add(_s(ab))
            except Exception:
                pass
    return ", ".join(sorted(codes)), ", ".join(sorted(areas)), ", ".join(sorted(abbrevs)), codes

# ----------------------- ISSN → Source ID (optional) -------------------

def fetch_source_id_for_issn(issn: str) -> Optional[str]:
    issn = _norm_issn(issn)
    if not issn:
        return None
    try:
        res = SerialTitle(issn)
        items: Iterable[Any]
        try:
            items = list(res) if isinstance(res, (list, tuple)) else [res]
        except Exception:
            items = [res]
        for it in items:
            for attr in ("source_id", "sourcerecord_id", "sourceid"):
                if hasattr(it, attr):
                    sid = _s(getattr(it, attr))
                    if sid.isdigit():
                        return sid
        return None
    except Exception as e:
        warnings.warn(f"SerialTitle lookup failed for ISSN {issn}: {e}")
        return None


def build_cs_by_source(cs_table: pd.DataFrame, serial_sleep: float = 0.1) -> pd.DataFrame:
    """Ensure a cs_by_source table with 'source_id' present, mapping from cs_table.
    If cs_table already has source_id, just dedupe and return.
    Otherwise, look up source_id from ISSN/e-ISSN via SerialTitle.
    """
    if "source_id" in cs_table.columns:
        df = cs_table.copy()
    else:
        df = cs_table.copy()
        df["source_id_from_print"] = df["issn_key"].apply(fetch_source_id_for_issn)
        if serial_sleep:
            time.sleep(serial_sleep)
        df["source_id_from_e"] = df["eissn_key"].apply(fetch_source_id_for_issn)
        df["source_id"] = df["source_id_from_print"].where(df["source_id_from_print"].notna(), df["source_id_from_e"])
        df = df.drop(columns=["source_id_from_print", "source_id_from_e"], errors="ignore")
    df = df[df["source_id"].astype(str).str.strip().ne("")].copy()
    df = df.drop_duplicates(subset=["source_id", "asjc_set"], keep="first")
    return df[["source_id", "asjc_set", "cs_percentile", "citescore"]]

# ----------------------- Matching logic --------------------------------

def _pick_best_candidate(cands: pd.DataFrame, article_asjc: Set[str]) -> Tuple[Optional[float], Optional[float]]:
    """Choose best row: prefer ASJC overlap, then highest percentile, else first.
    Returns (cs_percentile, citescore).
    """
    if cands is None or cands.empty:
        return None, None
    over = []
    for _i, row in cands.iterrows():
        cs_set = row.get("asjc_set") or set()
        if isinstance(cs_set, (set, frozenset)):
            cs_set2 = set(cs_set)
        elif isinstance(cs_set, (list, tuple)):
            cs_set2 = {str(x) for x in cs_set}
        else:
            cs_set2 = _norm_asjc_codes(cs_set)
        over.append(len(article_asjc & cs_set2))
    cands = cands.copy()
    cands["_overlap"] = over
    cands["_pct"] = cands["cs_percentile"].fillna(-1e9)
    cands = cands.sort_values(["_overlap", "_pct"], ascending=[False, False])
    top = cands.iloc[0]
    return top.get("cs_percentile"), top.get("citescore")


def enrich_with_citescore_sourceid_asjc(
    df_articles: pd.DataFrame,
    cs_table: pd.DataFrame,
    cs_by_source: pd.DataFrame,
) -> pd.DataFrame:
    if df_articles is None or df_articles.empty:
        return df_articles.copy()

    df = df_articles.copy()
    for col in ("issn_print", "issn_electronic", "source_id"):
        if col not in df.columns:
            df[col] = ""

    df["issn_key"] = df["issn_print"].astype(str).map(_norm_issn)
    df["eissn_key"] = df["issn_electronic"].astype(str).map(_norm_issn)

    a_asjc_sets: List[Set[str]] = []
    for v in df.get("asjc_codes", pd.Series([""] * len(df))):
        a_asjc_sets.append(_norm_asjc_codes(v))

    cs_p = cs_table[["issn_key", "asjc_set", "cs_percentile", "citescore"]].drop_duplicates()
    cs_e = cs_table[["eissn_key", "asjc_set", "cs_percentile", "citescore"]].drop_duplicates()

    out_pct: List[Optional[float]] = []
    out_val: List[Optional[float]] = []

    for idx, row in df.iterrows():
        article_asjc = a_asjc_sets[idx] if idx < len(a_asjc_sets) else set()
        sid = _s(row.get("source_id"))
        pct = None; val = None
        if sid:
            cands = cs_by_source[cs_by_source["source_id"].astype(str) == sid]
            pct, val = _pick_best_candidate(cands, article_asjc)
        if pct is None and val is None:
            issn = _s(row.get("issn_key"))
            eissn = _s(row.get("eissn_key"))
            cands = pd.concat([
                cs_p[cs_p["issn_key"] == issn],
                cs_e[cs_e["eissn_key"] == eissn],
            ], ignore_index=True)
            pct, val = _pick_best_candidate(cands, article_asjc)
        out_pct.append(pct)
        out_val.append(val)

    df["cs_percentile"] = out_pct
    df["citescore"] = out_val
    df["quartile"] = df["cs_percentile"].apply(quartile_from_percentile)
    return df

# ----------------------- Scopus collectors (with per-author aff detection) -------

def get_author_name(author_id: str) -> str:
    try:
        ar = AuthorRetrieval(author_id)
        name = f"{_s(ar.given_name)} {_s(ar.surname)}".strip()
        return name or author_id
    except Exception:
        return author_id

def get_author_eids(author_id: str) -> List[str]:
    try:
        s = ScopusSearch(f"AU-ID({author_id})", subscriber=True)
        return s.get_eids() or []
    except Exception as e:
        warnings.warn(f"ScopusSearch failed for AU-ID({author_id}): {e}")
        return []

def _safe_int(v) -> Optional[int]:
    try:
        return int(v)
    except Exception:
        return None

def get_article_metadata(eid: str, target_auid: Optional[str] = None):
    """
    Return article metadata dict. Also detect whether the given target_auid
    appears in the article and whether that author entry has affiliation_id == AFF_ID.
    """
    try:
        ar = AbstractRetrieval(eid, view="FULL")
    except ScopusException as e:
        warnings.warn(f"AbstractRetrieval failed for {eid}: {e}")
        return None

    if ar.subtype not in ("ar", "re"):
        return None

    year = ""
    if getattr(ar, "coverDate", None):
        year = _s(ar.coverDate)[:4]

    issn_print, issn_elec = _extract_issns(ar)
    asjc_codes_csv, asjc_areas_csv, asjc_abbrevs_csv, _codes_set = _extract_asjc(ar)

    bau_flag = False
    bau_orgs = []
    auid_org_map: Dict[str, str] = {}
    auth_err = None
    groups = getattr(ar, "authorgroup", None)
    if groups:
        try:
            for g in groups:
                try:
                    g_auid = str(getattr(g, "auid", "") or "")
                    org_val = getattr(g, "organization", None)
                    aff_id_val = _safe_int(getattr(g, "affiliation_id", None))
                    if g_auid:
                        auid_org_map[g_auid] = _s(org_val)
                    if target_auid and g_auid == str(target_auid) and AFF_ID and aff_id_val == _safe_int(AFF_ID):
                        bau_flag = True
                        if org_val:
                            bau_orgs.append(_s(org_val))
                except Exception:
                    continue
        except Exception as ex:
            auth_err = str(ex)
    else:
        try:
            authors = getattr(ar, "authors", None) or []
            for a in authors:
                try:
                    g_auid = str(getattr(a, "auid", "") or "")
                    org_val = getattr(a, "orgname", None) or getattr(a, "affiliation", None) or ""
                    if g_auid:
                        auid_org_map[g_auid] = _s(org_val)
                except Exception:
                    continue
        except Exception:
            pass

    insts = getattr(ar, "affiliation", None) or []

    return {
        "eid": _s(eid),
        "title": _s(ar.title),
        "year": _s(year),
        "publication_name": _s(getattr(ar, "publicationName", "")),
        "subtype": _s(ar.subtype),
        "doi": _s(getattr(ar, "doi", "")),
        "source_id": _s(getattr(ar, "source_id", "")),
        "issn_print": _s(issn_print),
        "issn_electronic": _s(issn_elec),
        "asjc_codes": asjc_codes_csv,
        "asjc_areas": asjc_areas_csv,
        "asjc_abbrevs": asjc_abbrevs_csv,
        "authors_count": len(ar.authors) if getattr(ar, "authors", None) else 1,
        "combined": "; ".join([t for t in (getattr(ar, "authkeywords", []) or []) if _s(t)]),
        "abstract": _s(getattr(ar, "description", "")),
        "author_org_map_json": json.dumps(auid_org_map, ensure_ascii=False),
        "is_bau_author": bau_flag,
        "bau_author_orgs": "; ".join(dict.fromkeys(bau_orgs)),
        "affiliations_list": [(getattr(i, "name", "") if hasattr(i, "name") else _s(i)) for i in insts],
        "auth_group_error": auth_err,
    }

# ----------------------- APP calculation helpers -----------------------

def _qc_from_percentile(p: Optional[float]) -> Optional[float]:
    if p is None:
        return None
    try:
        p = float(p)
    except Exception:
        return None
    if p >= 90:   # Top 10%
        return 1.4
    if p >= 75:   # Q1
        return 1.0
    if p >= 50:   # Q2
        return 0.8
    if p >= 25:   # Q3
        return 0.6
    if p >= 0:    # Q4
        return 0.4
    return None

def _ac_from_authors(n: Any) -> float:
    try:
        n = int(n)
    except Exception:
        n = 1
    return 1.2 if n <= 1 else 1.2 / max(n, 1)

def _to_int_year(y: Any) -> Optional[int]:
    try:
        s = str(y)
        m = re.search(r"\d{4}", s)
        return int(m.group(0)) if m else None
    except Exception:
        return None

def build_app_sheet(df_articles: pd.DataFrame, now_year: Optional[int] = None) -> Tuple[pd.DataFrame, Dict[str, Any]]:
    if df_articles is None or df_articles.empty:
        return pd.DataFrame(), {"app_total": 0.0, "eligibility": "No eligible items", "years": []}

    cy = now_year or datetime.now().year
    years_ok = {cy, cy - 1, cy - 2}

    tmp = df_articles.copy()
    tmp["year_i"] = tmp.get("year", "").apply(_to_int_year)
    tmp["cs_percentile_num"] = pd.to_numeric(tmp.get("cs_percentile"), errors="coerce")
    tmp["authors_count_i"] = pd.to_numeric(tmp.get("authors_count"), errors="coerce").fillna(1).astype(int)
    tmp["subtype_norm"] = tmp.get("subtype", "").astype(str).str.lower()

    eligible = tmp[
        tmp["year_i"].isin(years_ok) &
        tmp["cs_percentile_num"].notna() &
        (tmp["subtype_norm"] == "ar")
    ].copy()

    if eligible.empty:
        return pd.DataFrame(), {"app_total": 0.0, "eligibility": "No eligible items (no 'ar' articles in window)", "years": sorted(years_ok)}

    eligible["QC"] = eligible["cs_percentile_num"].apply(_qc_from_percentile)
    eligible["AC"] = eligible["authors_count_i"].apply(_ac_from_authors)
    eligible = eligible[eligible["QC"].notna()].copy()
    if eligible.empty:
        return pd.DataFrame(), {"app_total": 0.0, "eligibility": "No eligible items (QC missing)", "years": sorted(years_ok)}

    eligible["Contribution"] = eligible["AC"] * eligible["QC"]
    eligible["AC"] = eligible["AC"].round(2)
    eligible["QC"] = eligible["QC"].round(2)
    eligible["Contribution"] = eligible["Contribution"].round(2)

    out_cols = [
        "eid", "title", "year", "publication_name",
        "authors_count", "cs_percentile", "quartile",
        "AC", "QC", "Contribution"
    ]
    for c in out_cols:
        if c not in eligible.columns:
            eligible[c] = pd.Series(dtype="object")
    df_app = eligible[out_cols].sort_values(["year", "Contribution"], ascending=[False, False]).reset_index(drop=True)

    app_total = float(df_app["Contribution"].sum().round(2))

    if app_total > 1.0:
        elig = "APP > 1.0 → up to 2 supports / AY (only 1 requires full indexing & APP check)"
    elif app_total >= 0.4:
        elig = "0.4 ≤ APP ≤ 1.0 → 1 support / AY"
    else:
        elig = "APP < 0.4 → 1 support / AY (if other criteria met)"

    summary = {"app_total": round(app_total, 2), "eligibility": elig, "years": sorted(years_ok)}
    return df_app, summary

# ----------------------- Excel writer (Articles / BAU Articles / APP) -----------------

def _write_excel_xlsxwriter(articles_df: pd.DataFrame, bau_df: pd.DataFrame, path: Path, app_df: Optional[pd.DataFrame] = None, app_summary: Optional[Dict[str, Any]] = None):
    with pd.ExcelWriter(path, engine="xlsxwriter") as xl:
        # Articles (ALL)
        articles_df.to_excel(xl, sheet_name="Articles", index=False)
        ws = xl.sheets["Articles"]
        try:
            ws.freeze_panes(1, 0)
            ws.set_column("A:A", 20)  # eid
            ws.set_column("B:B", 50)  # title
            ws.set_column("C:C", 8)   # year
            ws.set_column("D:D", 36)  # publication_name
            ws.set_column("E:E", 8)   # subtype
            ws.set_column("F:F", 26)  # doi
            ws.set_column("G:G", 14)  # source_id
            ws.set_column("H:I", 14)  # ISSNs
            ws.set_column("J:J", 18)  # asjc_codes
            ws.set_column("K:K", 26)  # asjc_areas
            ws.set_column("L:L", 14)  # asjc_abbrevs
            ws.set_column("M:N", 14)  # cs_percentile / citescore
            ws.set_column("O:O", 10)  # quartile
            ws.set_column("P:P", 12)  # authors_count
            ws.set_column("Q:Q", 28)  # combined
            ws.set_column("R:R", 80)  # abstract
            ws.set_column("S:U", 28)  # affiliation fields
        except Exception:
            pass

        # BAU Articles
        bau_df.to_excel(xl, sheet_name="BAU Articles", index=False)
        ws2 = xl.sheets["BAU Articles"]
        try:
            ws2.freeze_panes(1, 0)
            ws2.set_column("A:A", 20)
            ws2.set_column("B:B", 50)
            ws2.set_column("C:C", 8)
            ws2.set_column("D:D", 36)
            ws2.set_column("E:E", 8)
            ws2.set_column("F:F", 26)
            ws2.set_column("G:G", 14)
            ws2.set_column("H:I", 14)
            ws2.set_column("J:J", 18)
            ws2.set_column("K:K", 26)
            ws2.set_column("L:L", 14)
            ws2.set_column("M:N", 14)
            ws2.set_column("O:O", 10)
            ws2.set_column("P:P", 12)
            ws2.set_column("Q:Q", 28)
            ws2.set_column("R:R", 80)
            ws2.set_column("S:U", 28)
        except Exception:
            pass

        # APP sheet (uses BAU Articles)
        if app_df is not None and not app_df.empty:
            ws3 = xl.book.add_worksheet("APP")
            xl.sheets["APP"] = ws3
            try:
                ws3.write(0, 0, "APP calculation — last 3 calendar years (journal articles only; subtype == 'ar')")
                if app_summary:
                    ws3.write(1, 0, "Years considered")
                    ws3.write(1, 1, ", ".join(str(y) for y in app_summary.get("years", [])))
                    ws3.write(2, 0, "APP Score")
                    ws3.write(2, 1, app_summary.get("app_total", 0.0))
                    ws3.write(3, 0, "Eligibility")
                    ws3.write(3, 1, app_summary.get("eligibility", ""))
                (app_df.reset_index(drop=True)).to_excel(xl, sheet_name="APP", index=False, startrow=5)
                ws3.freeze_panes(6, 0)
                ws3.set_column("A:A", 20)
                ws3.set_column("B:B", 60)
                ws3.set_column("C:C", 8)
                ws3.set_column("D:D", 36)
                ws3.set_column("E:E", 12)
                ws3.set_column("F:F", 14)
                ws3.set_column("G:G", 10)
                ws3.set_column("H:J", 14)
            except Exception:
                try:
                    app_df.to_excel(xl, sheet_name="APP", index=False)
                except Exception:
                    pass

def _write_excel_openpyxl(articles_df: pd.DataFrame, bau_df: pd.DataFrame, path: Path, app_df: Optional[pd.DataFrame] = None, app_summary: Optional[Dict[str, Any]] = None):
    with pd.ExcelWriter(path, engine="openpyxl") as xl:
        articles_df.to_excel(xl, sheet_name="Articles", index=False)
        bau_df.to_excel(xl, sheet_name="BAU Articles", index=False)
        try:
            if app_df is not None and not app_df.empty:
                app_df.to_excel(xl, sheet_name="APP", index=False, startrow=5)
        except Exception:
            try:
                app_df.to_excel(xl, sheet_name="APP", index=False)
            except Exception:
                pass

def save_author_excel(author_id: str, author_name: str, articles_df: pd.DataFrame, bau_df: pd.DataFrame, out_dir: Path) -> str:
    out_dir.mkdir(parents=True, exist_ok=True)
    path = out_dir / make_author_filename(author_name, author_id)

    need = [
        "eid","title","year","publication_name","subtype","doi",
        "source_id","issn_print","issn_electronic",
        "asjc_codes","asjc_areas","asjc_abbrevs",
        "cs_percentile","citescore","quartile",
        "authors_count","combined","abstract",
        "is_bau_author","bau_author_orgs","author_org_map_json"
    ]

    # Ensure columns present for both frames
    art = articles_df.copy() if articles_df is not None else pd.DataFrame()
    bau = bau_df.copy() if bau_df is not None else pd.DataFrame()
    for df in (art, bau):
        for c in need:
            if c not in df.columns:
                df[c] = pd.Series(dtype="object")
    art = art[need]
    bau = bau[need]

    # APP computed from BAU articles
    app_df, app_summary = build_app_sheet(bau)

    try:
        _write_excel_xlsxwriter(art, bau, path, app_df=app_df if not app_df.empty else None, app_summary=app_summary)
    except Exception:
        _write_excel_openpyxl(art, bau, path, app_df=app_df if not app_df.empty else None, app_summary=app_summary)
    return str(path)

# ----------------------- Orchestration --------------------------------

def process_author(author_id: str, cs_table: pd.DataFrame, out_dir: Path, sleep: float, serial_sleep: float, cs_by_source: Optional[pd.DataFrame] = None) -> Optional[str]:
    try:
        name = get_author_name(author_id)
        print(f"→ AU-ID {author_id} — {name} (AFF_ID filter: {AFF_ID})")
        eids = get_author_eids(author_id)
        recs_all: List[Dict[str, Any]] = []
        recs_bau: List[Dict[str, Any]] = []

        for eid in eids:
            md = get_article_metadata(eid, target_auid=author_id)
            if md:
                rec = {
                    "eid": md.get("eid"),
                    "title": md.get("title"),
                    "year": md.get("year"),
                    "publication_name": md.get("publication_name"),
                    "subtype": md.get("subtype"),
                    "doi": md.get("doi"),
                    "source_id": md.get("source_id"),
                    "issn_print": md.get("issn_print"),
                    "issn_electronic": md.get("issn_electronic"),
                    "asjc_codes": md.get("asjc_codes"),
                    "asjc_areas": md.get("asjc_areas"),
                    "asjc_abbrevs": md.get("asjc_abbrevs"),
                    "authors_count": md.get("authors_count"),
                    "combined": md.get("combined"),
                    "abstract": md.get("abstract"),
                    "is_bau_author": md.get("is_bau_author"),
                    "bau_author_orgs": md.get("bau_author_orgs"),
                    "author_org_map_json": md.get("author_org_map_json"),
                }
                # add to "all articles" unconditionally
                recs_all.append(rec)
                # add to BAU-only if author-affiliated on this record
                if md.get("is_bau_author", False):
                    recs_bau.append(rec)
            if sleep:
                time.sleep(sleep)

        df_all = pd.DataFrame(recs_all)
        df_bau = pd.DataFrame(recs_bau)

        if not df_all.empty:
            if cs_by_source is None:
                cs_by_source = build_cs_by_source(cs_table, serial_sleep=serial_sleep)
            df_all = enrich_with_citescore_sourceid_asjc(df_all, cs_table, cs_by_source)

        if not df_bau.empty:
            if cs_by_source is None:
                cs_by_source = build_cs_by_source(cs_table, serial_sleep=serial_sleep)
            df_bau = enrich_with_citescore_sourceid_asjc(df_bau, cs_table, cs_by_source)

        path = save_author_excel(author_id, name, df_all if not df_all.empty else pd.DataFrame(), df_bau if not df_bau.empty else pd.DataFrame(), out_dir)
        print(f"   ✓ Wrote {Path(path).name}  (Articles: {len(df_all) if df_all is not None else 0}, BAU Articles: {len(df_bau) if df_bau is not None else 0})")
        return path
    except Exception as e:
        print(f"   ⚠️ Failed {author_id}: {e}")
        return None

# ----------------------- Public API (Jupyter) -------------------------

def run(
    auids: Optional[str] = None,
    citescore: Optional[str] = None,
    outdir: str = "authors",
    sleep: float = 0.05,
    serial_sleep: float = 0.1,
    no_prompt: bool = True,
    aff_id: Optional[str] = None,
) -> None:
    global AFF_ID
    if aff_id is not None:
        AFF_ID = aff_id

    out_dir = Path(outdir)
    cs_path = resolve_citescore_path(citescore, no_prompt=no_prompt)
    if not cs_path:
        raise FileNotFoundError(
            "Could not locate CiteScore file. Pass citescore=..., set CITESCORE_CSV env var, or place the file in a default folder."
        )
    print(f"Using CiteScore file: {cs_path}")
    cs_table = load_citescore_table(cs_path)

    ids = read_auids_from_cli_or_file(auids)
    if not ids:
        print("No AU-IDs provided. Provide auids='555...,572...' or authors.txt file.")
        return
    print(f"CiteScore rows: {len(cs_table)} | Authors: {len(ids)} | Affiliation filter: {AFF_ID}")

    cs_by_source = build_cs_by_source(cs_table, serial_sleep=serial_sleep)

    out_dir.mkdir(parents=True, exist_ok=True)
    for i, auid in enumerate(ids, 1):
        print(f"[{i}/{len(ids)}]")
        process_author(auid, cs_table, out_dir, sleep=sleep, serial_sleep=serial_sleep, cs_by_source=cs_by_source)
    print(f"\nDone. Files saved to: {out_dir.resolve()}")

# ----------------------- CLI -----------------------------------------

def main(argv: Optional[list] = None):
    global AFF_ID
    ap = argparse.ArgumentParser(description="Fetch Scopus pubs by AU-ID and match CiteScore by Source ID then ASJC (ISSN fallback). Produces 'Articles', 'BAU Articles' and 'APP' sheets per author.")
    ap.add_argument("--auids", type=str, default=None, help="Comma/space-separated Scopus Author IDs. If omitted, reads authors.txt")
    ap.add_argument("--citescore", type=str, default=None, help="Path to CiteScore CSV/XLSX, or a folder containing it")
    ap.add_argument("--outdir", type=str, default="authors", help="Output directory (default: ./authors)")
    ap.add_argument("--sleep", type=float, default=0.05, help="Sleep between EID fetches (seconds)")
    ap.add_argument("--serial-sleep", type=float, default=0.1, help="Sleep between SerialTitle ISSN lookups (seconds)")
    ap.add_argument("--no-prompt", action="store_true", help="Do not prompt for missing CiteScore path; exit with error")
    ap.add_argument("--aff-id", type=str, default=None, help=f"Optional affiliation ID to isolate author-affiliated publications (default: {AFF_ID_DEFAULT})")
    args, _unknown = ap.parse_known_args(argv)

    if args.aff_id is not None:
        AFF_ID = args.aff_id

    cs_path = resolve_citescore_path(args.citescore, no_prompt=args.no_prompt)
    if not cs_path:
        if _is_ipython() and not args.no_prompt:
            try:
                entered = input("Enter CiteScore CSV/XLSX path or containing folder: ").strip('"').strip()
            except EOFError:
                entered = ""
            if entered:
                cs_path = resolve_citescore_path(entered, no_prompt=True)
        if not cs_path:
            print("❌ Could not locate the CiteScore file. Pass --citescore or set CITESCORE_CSV.")
            if _is_ipython():
                return
            sys.exit(2)
    print(f"Using CiteScore file: {cs_path}")
    cs_table = load_citescore_table(cs_path)

    auids = read_auids_from_cli_or_file(args.auids)
    if not auids and _is_ipython():
        try:
            entered = input("Enter Scopus Author IDs (comma or space separated), or leave blank to cancel: ").strip()
        except EOFError:
            entered = ""
        if entered:
            auids = read_auids_from_cli_or_file(entered)
    if not auids:
        print("No AU-IDs provided. Use --auids, or create authors.txt with one AU-ID per line.")
        if _is_ipython():
            return
        sys.exit(0)

    out_dir = Path(args.outdir)
    print(f"CiteScore rows: {len(cs_table)} | Authors: {len(auids)} | Out: {out_dir.resolve()} | Affiliation filter: {AFF_ID}")
    out_dir.mkdir(parents=True, exist_ok=True)

    cs_by_source = build_cs_by_source(cs_table, serial_sleep=args.serial_sleep)

    written = []
    for i, auid in enumerate(auids, 1):
        print(f"[{i}/{len(auids)}] Processing AU-ID {auid} … (AFF_ID={AFF_ID})")
        p = process_author(auid, cs_table, out_dir, sleep=args.sleep, serial_sleep=args.serial_sleep, cs_by_source=cs_by_source)
        if p:
            written.append(p)

    print(f"\nDone. Wrote {len(written)} file(s) to {out_dir.resolve()}.")

if __name__ == "__main__":
    main()


Using CiteScore file: C:\Users\yusef.atteyih\Desktop\Academic Research Unit\Yusef ATTEYIH\Data Solutions\Data Solutions 2.0\APP Calculation\CiteScore 2024\CiteScore 2024 annual values.csv


Enter Scopus Author IDs (comma or space separated), or leave blank to cancel:  57193254610


CiteScore rows: 75679 | Authors: 1 | Out: C:\Users\yusef.atteyih\Desktop\Academic Research Unit\Yusef ATTEYIH\Data Solutions\Data Solutions 2.0\APP Calculation\authors | Affiliation filter: 60021379
[1/1] Processing AU-ID 57193254610 … (AFF_ID=60021379)
→ AU-ID 57193254610 — Fadime İrem Doğan (AFF_ID filter: 60021379)
   ✓ Wrote _fadime_irem_dogan___57193254610.xlsx  (Articles: 5, BAU Articles: 3)

Done. Wrote 1 file(s) to C:\Users\yusef.atteyih\Desktop\Academic Research Unit\Yusef ATTEYIH\Data Solutions\Data Solutions 2.0\APP Calculation\authors.


In [38]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
r"""
Scopus → CiteScore with disambiguation by **Source ID then ASJC (sub-subject area)**

This version:
• Stores ASJC sets as frozenset (hashable).
• Matches publications to CiteScore by Source ID then ASJC (ISSN fallback).
• Performs per-article author→affiliation mapping and isolates items where the
  author is affiliated with the configured AFF_ID on that paper.
• Produces three sheets in each author workbook:
    - "Articles"      : ALL articles for the author (no affiliation filtering)
    - "BAU Articles"  : only those articles where the author is affiliated with AFF_ID
    - "APP"           : Participation Score computed from BAU Articles (subtype == "ar")
• APP uses Author Coefficient (AC = 1.2 if sole author else 1.2 / n_authors)
  and Quartile Coefficient (QC) mapped from CiteScore percentile,
  Contribution = AC × QC, summed and rounded.
• NOTE: APP eligibility and support thresholds are expressed **per academic year (AY)**.

Usage:
  - Jupyter: run(auids="12345,67890", citescore="path/to/CiteScore.csv", aff_id="60021379")
  - CLI: python this_script.py --auids 57193254610 --citescore "CiteScore 2024.csv" --aff-id 60021379

Requirements:
  pybliometrics, pandas, xlsxwriter or openpyxl
  Configure pybliometrics Scopus API credentials
"""

from __future__ import annotations

import argparse
import json
import os
import re
import sys
import time
import unicodedata
import warnings
from datetime import datetime
from pathlib import Path
from typing import Any, Dict, Iterable, List, Optional, Set, Tuple

import pandas as pd

# --- Scopus (pybliometrics) ---
try:
    from pybliometrics.scopus import AbstractRetrieval, AuthorRetrieval, ScopusSearch, SerialTitle
    from pybliometrics.scopus.exception import ScopusException
except Exception as e:
    raise RuntimeError(
        "pybliometrics is required. Install with `pip install pybliometrics` "
        "and ensure your Scopus API credentials are configured."
    ) from e

# ----------------------- IPython detection ----------------------------

def _is_ipython() -> bool:
    return ("ipykernel" in sys.modules) or ("IPython" in sys.modules)

# ----------------------- Defaults & Affiliation filter ----------------

AFF_ID_DEFAULT = "60021379"
AFF_ID: Optional[str] = AFF_ID_DEFAULT

DEFAULT_USER_CITESCORE_DIRS = [
    Path(r"C:\\Users\\yusef.atteyih\\Desktop\\Academic Research Unit\\Yusef ATTEYIH\\Data Solutions\\Data Solutions 2.0\\APP Calculation\\CiteScore 2024"),
]
DEFAULT_CITESCORE_DIRS = [Path("CiteScore 2024"), Path(".")]
POSSIBLE_CS_FILENAMES = [
    "CiteScore 2024 annual values.csv",
    "CiteScore 2024 annual values.xlsx",
    "CiteScore 2024.csv",
    "citescore.csv",
    "citescore.xlsx",
]

# ----------------------- Small helpers --------------------------------

def _s(x) -> str:
    try:
        if x is None or (isinstance(x, float) and pd.isna(x)):
            return ""
    except Exception:
        pass
    return x if isinstance(x, str) else str(x)

def _norm_issn(s: str) -> str:
    """Uppercase; keep 0-9 and 'X'; strip others."""
    return re.sub(r"[^0-9X]", "", _s(s).upper())

def _norm_asjc_codes(raw: Any) -> Set[str]:
    """Return a set of 4-digit ASJC codes as strings."""
    out: Set[str] = set()
    if raw is None:
        return out
    if isinstance(raw, (list, tuple, set)):
        it = raw
    else:
        it = re.split(r"[^0-9]", _s(raw))
    for tok in it:
        if tok and tok.isdigit():
            if len(tok) == 4:
                out.add(tok)
            elif len(tok) > 4:
                out.add(tok[-4:])
    return out

def _coerce_percentile(val) -> Optional[float]:
    txt = _s(val).strip()
    if not txt:
        return None
    txt = txt.replace("%", "").replace(" ", "").replace(",", ".")
    m = re.search(r"[-+]?\d*\.?\d+", txt)
    if not m:
        return None
    try:
        return float(m.group(0))
    except Exception:
        return None

def _coerce_float(val) -> Optional[float]:
    txt = _s(val).strip().replace(",", ".")
    if not txt:
        return None
    m = re.search(r"[-+]?\d*\.?\d+", txt)
    if not m:
        return None
    try:
        return float(m.group(0))
    except Exception:
        return None

def quartile_from_percentile(p: Optional[float]) -> str:
    if p is None:
        return ""
    try:
        p = float(p)
    except Exception:
        return ""
    if p >= 90: return "QT"
    if p >= 75: return "Q1"
    if p >= 50: return "Q2"
    if p >= 25: return "Q3"
    return "Q4"

# ----------------------- Filenames ------------------------------------

def _ascii_slug(s: str) -> str:
    s = unicodedata.normalize("NFKD", _s(s)).encode("ascii", "ignore").decode("ascii")
    s = s.lower()
    s = re.sub(r"[^0-9a-z]+", "_", s)
    s = re.sub(r"_+", "_", s).strip("_")
    return s

def make_author_filename(author_name: str, author_id: str) -> str:
    name_slug = _ascii_slug(author_name)
    leading = "_" + name_slug if not name_slug.startswith("_") else name_slug
    return f"{leading}___{author_id}.xlsx"

# ----------------------- Read AU-IDs ----------------------------------

author_id_file_candidates = ("authors.txt", "auids.txt", "ScopusAuthorIDs.txt")

def read_auids_from_cli_or_file(cli_auids: Optional[str]) -> List[str]:
    ids: List[str] = []
    if cli_auids:
        for tok in re.split(r"[,\s]+", cli_auids.strip()):
            if tok and tok.isdigit():
                ids.append(tok)
    if ids:
        return sorted(set(ids))
    for fname in author_id_file_candidates:
        p = Path(fname)
        if p.exists():
            try:
                raw = p.read_text(encoding="utf-8")
            except Exception:
                raw = p.read_text(errors="ignore")
            for line in raw.splitlines():
                t = line.split("#", 1)[0].strip()
                if t.isdigit():
                    ids.append(t)
            if ids:
                return sorted(set(ids))
    return []

# ----------------------- CiteScore path --------------------------------

def _candidate_files_in_dir(d: Path) -> List[Path]:
    return [p for name in POSSIBLE_CS_FILENAMES if (p := d / name).exists() and p.is_file()]

def resolve_citescore_path(arg: Optional[str], no_prompt: bool = False) -> Optional[Path]:
    if arg:
        p = Path(arg)
        if p.exists():
            if p.is_file(): return p
            if p.is_dir():
                cand = _candidate_files_in_dir(p)
                if cand: return cand[0]
    envp = os.environ.get("CITESCORE_CSV")
    if envp:
        p = Path(envp)
        if p.exists() and p.is_file(): return p
    for d in DEFAULT_USER_CITESCORE_DIRS + DEFAULT_CITESCORE_DIRS:
        if d.exists() and d.is_dir():
            cand = _candidate_files_in_dir(d)
            if cand: return cand[0]
    if not no_prompt:
        try:
            path_in = input("Path to CiteScore CSV/XLSX (or folder containing it): ").strip('"').strip()
            if path_in:
                p = Path(path_in)
                if p.exists():
                    if p.is_file(): return p
                    if p.is_dir():
                        cand = _candidate_files_in_dir(p)
                        if cand: return cand[0]
        except EOFError:
            pass
    return None

# ----------------------- Read CiteScore table --------------------------

def robust_read_table(path: Path) -> pd.DataFrame:
    if not path.exists():
        raise FileNotFoundError(f"CiteScore file not found: {path}")
    suffix = path.suffix.lower()
    if suffix in (".xlsx", ".xls"):
        return pd.read_excel(path)
    encodings = ["utf-8-sig", "utf-16", "utf-16le", "utf-16be", "cp1254", "iso-8859-9", "cp1252", "latin1"]
    last_err = None
    for enc in encodings:
        try:
            return pd.read_csv(path, encoding=enc, engine="python", sep=None)
        except Exception as e:
            last_err = e
            continue
    try:
        return pd.read_csv(path, encoding="latin1", engine="python", sep=None, on_bad_lines="skip")
    except TypeError:
        return pd.read_csv(path, encoding="latin1", engine="python", sep=None, error_bad_lines=False)  # type: ignore

def load_citescore_table(path: Path) -> pd.DataFrame:
    """
    Return columns:
      source_id (if present), issn_key, eissn_key, asjc_set (frozenset), cs_percentile, citescore
    """
    cs = robust_read_table(path)
    norm = {c.strip().lower(): c for c in cs.columns}

    p = norm.get("print issn") or norm.get("p-issn") or norm.get("issn")
    e = norm.get("e-issn") or norm.get("eissn")
    pct = norm.get("percentile") or norm.get("citescore percentile")
    val = norm.get("citescore") or norm.get("citescore 2024")
    src = norm.get("source id") or norm.get("scopus source id") or norm.get("scopus sourceid")
    asjc_col = norm.get("asjc") or norm.get("asjc code") or norm.get("asjc codes") or norm.get("subject area asjc")

    if not all([p, e, pct, val]):
        raise KeyError(
            f"CiteScore table must include 'Print ISSN', 'E-ISSN', 'Percentile', 'CiteScore'. Found: {list(cs.columns)}"
        )

    # Standardize names
    cs = cs.rename(columns={p: "print_issn", e: "e_issn", pct: "cs_percentile", val: "citescore"})
    if src:
        cs = cs.rename(columns={src: "source_id"})
    if asjc_col:
        cs = cs.rename(columns={asjc_col: "asjc_raw"})
    else:
        cs["asjc_raw"] = ""

    # Coerce types
    cs["print_issn"] = cs["print_issn"].astype(str)
    cs["e_issn"] = cs["e_issn"].astype(str)
    cs["cs_percentile"] = cs["cs_percentile"].apply(_coerce_percentile)
    cs["citescore"] = cs["citescore"].apply(_coerce_float)
    if "source_id" in cs.columns:
        cs["source_id"] = cs["source_id"].astype(str).str.extract(r"(\d+)", expand=False).fillna("")

    # Normalized keys
    cs["issn_key"] = cs["print_issn"].map(_norm_issn)
    cs["eissn_key"] = cs["e_issn"].map(_norm_issn)

    # ASJC as frozenset (hashable)
    cs["asjc_set"] = cs["asjc_raw"].apply(lambda v: frozenset(_norm_asjc_codes(v)))

    # Keep minimal columns
    keep = ["issn_key", "eissn_key", "asjc_set", "cs_percentile", "citescore"]
    if "source_id" in cs.columns:
        keep.insert(0, "source_id")
    cs = cs[keep]
    return cs

# ----------------------- Scopus helpers --------------------------------

def _extract_issns(ar: Any) -> Tuple[str, str]:
    p = ""; e = ""
    if hasattr(ar, "eIssn"):
        e = _s(getattr(ar, "eIssn"))
    if not e and hasattr(ar, "e_issn"):
        e = _s(getattr(ar, "e_issn"))
    if hasattr(ar, "issn"):
        obj = getattr(ar, "issn")
        if isinstance(obj, str):
            if "ISSN(" in obj:
                mp = re.search(r"print\s*=\s*'([^']+)'", obj)
                me = re.search(r"electronic\s*=\s*'([^']+)'", obj)
                if mp: p = mp.group(1)
                if me and not e: e = me.group(1)
            else:
                p = _s(obj)
        else:
            try:
                p_obj = getattr(obj, "print", ""); e_obj = getattr(obj, "electronic", "")
                if p_obj: p = _s(p_obj)
                if not e and e_obj: e = _s(e_obj)
            except Exception:
                txt = _s(obj)
                mp = re.search(r"print\s*=\s*'([^']+)'", txt)
                me = re.search(r"electronic\s*=\s*'([^']+)'", txt)
                if mp: p = mp.group(1)
                if me and not e: e = me.group(1)
    return p, e


def _extract_asjc(ar: Any) -> Tuple[str, str, str, Set[str]]:
    codes: Set[str] = set(); areas: Set[str] = set(); abbrevs: Set[str] = set()
    sa = getattr(ar, "subject_areas", None)
    if sa:
        try:
            for it in sa:
                c = getattr(it, "code", None)
                a = getattr(it, "area", None)
                ab = getattr(it, "abbrev", None)
                if c is not None:
                    codes.add(f"{int(c):04d}" if str(c).isdigit() else str(c))
                if a: areas.add(_s(a))
                if ab: abbrevs.add(_s(ab))
        except Exception:
            try:  # dict-like
                for it in sa:
                    c = it.get("code"); a = it.get("area"); ab = it.get("abbrev")
                    if c is not None:
                        codes.add(f"{int(c):04d}" if str(c).isdigit() else str(c))
                    if a: areas.add(_s(a))
                    if ab: abbrevs.add(_s(ab))
            except Exception:
                pass
    return ", ".join(sorted(codes)), ", ".join(sorted(areas)), ", ".join(sorted(abbrevs)), codes

# ----------------------- ISSN → Source ID (optional) -------------------

def fetch_source_id_for_issn(issn: str) -> Optional[str]:
    issn = _norm_issn(issn)
    if not issn:
        return None
    try:
        res = SerialTitle(issn)
        items: Iterable[Any]
        try:
            items = list(res) if isinstance(res, (list, tuple)) else [res]
        except Exception:
            items = [res]
        for it in items:
            for attr in ("source_id", "sourcerecord_id", "sourceid"):
                if hasattr(it, attr):
                    sid = _s(getattr(it, attr))
                    if sid.isdigit():
                        return sid
        return None
    except Exception as e:
        warnings.warn(f"SerialTitle lookup failed for ISSN {issn}: {e}")
        return None


def build_cs_by_source(cs_table: pd.DataFrame, serial_sleep: float = 0.1) -> pd.DataFrame:
    """Ensure a cs_by_source table with 'source_id' present, mapping from cs_table.
    If cs_table already has source_id, just dedupe and return.
    Otherwise, look up source_id from ISSN/e-ISSN via SerialTitle.
    """
    if "source_id" in cs_table.columns:
        df = cs_table.copy()
    else:
        df = cs_table.copy()
        df["source_id_from_print"] = df["issn_key"].apply(fetch_source_id_for_issn)
        if serial_sleep:
            time.sleep(serial_sleep)
        df["source_id_from_e"] = df["eissn_key"].apply(fetch_source_id_for_issn)
        df["source_id"] = df["source_id_from_print"].where(df["source_id_from_print"].notna(), df["source_id_from_e"])
        df = df.drop(columns=["source_id_from_print", "source_id_from_e"], errors="ignore")
    df = df[df["source_id"].astype(str).str.strip().ne("")].copy()
    df = df.drop_duplicates(subset=["source_id", "asjc_set"], keep="first")
    return df[["source_id", "asjc_set", "cs_percentile", "citescore"]]

# ----------------------- Matching logic --------------------------------

def _pick_best_candidate(cands: pd.DataFrame, article_asjc: Set[str]) -> Tuple[Optional[float], Optional[float]]:
    """Choose best row: prefer ASJC overlap, then highest percentile, else first.
    Returns (cs_percentile, citescore).
    """
    if cands is None or cands.empty:
        return None, None
    over = []
    for _i, row in cands.iterrows():
        cs_set = row.get("asjc_set") or set()
        if isinstance(cs_set, (set, frozenset)):
            cs_set2 = set(cs_set)
        elif isinstance(cs_set, (list, tuple)):
            cs_set2 = {str(x) for x in cs_set}
        else:
            cs_set2 = _norm_asjc_codes(cs_set)
        over.append(len(article_asjc & cs_set2))
    cands = cands.copy()
    cands["_overlap"] = over
    cands["_pct"] = cands["cs_percentile"].fillna(-1e9)
    cands = cands.sort_values(["_overlap", "_pct"], ascending=[False, False])
    top = cands.iloc[0]
    return top.get("cs_percentile"), top.get("citescore")


def enrich_with_citescore_sourceid_asjc(
    df_articles: pd.DataFrame,
    cs_table: pd.DataFrame,
    cs_by_source: pd.DataFrame,
) -> pd.DataFrame:
    if df_articles is None or df_articles.empty:
        return df_articles.copy()

    df = df_articles.copy()
    for col in ("issn_print", "issn_electronic", "source_id"):
        if col not in df.columns:
            df[col] = ""

    df["issn_key"] = df["issn_print"].astype(str).map(_norm_issn)
    df["eissn_key"] = df["issn_electronic"].astype(str).map(_norm_issn)

    a_asjc_sets: List[Set[str]] = []
    for v in df.get("asjc_codes", pd.Series([""] * len(df))):
        a_asjc_sets.append(_norm_asjc_codes(v))

    cs_p = cs_table[["issn_key", "asjc_set", "cs_percentile", "citescore"]].drop_duplicates()
    cs_e = cs_table[["eissn_key", "asjc_set", "cs_percentile", "citescore"]].drop_duplicates()

    out_pct: List[Optional[float]] = []
    out_val: List[Optional[float]] = []

    for idx, row in df.iterrows():
        article_asjc = a_asjc_sets[idx] if idx < len(a_asjc_sets) else set()
        sid = _s(row.get("source_id"))
        pct = None; val = None
        if sid:
            cands = cs_by_source[cs_by_source["source_id"].astype(str) == sid]
            pct, val = _pick_best_candidate(cands, article_asjc)
        if pct is None and val is None:
            issn = _s(row.get("issn_key"))
            eissn = _s(row.get("eissn_key"))
            cands = pd.concat([
                cs_p[cs_p["issn_key"] == issn],
                cs_e[cs_e["eissn_key"] == eissn],
            ], ignore_index=True)
            pct, val = _pick_best_candidate(cands, article_asjc)
        out_pct.append(pct)
        out_val.append(val)

    df["cs_percentile"] = out_pct
    df["citescore"] = out_val
    df["quartile"] = df["cs_percentile"].apply(quartile_from_percentile)
    return df

# ----------------------- Scopus collectors (with per-author aff detection) -------

def get_author_name(author_id: str) -> str:
    try:
        ar = AuthorRetrieval(author_id)
        name = f"{_s(ar.given_name)} {_s(ar.surname)}".strip()
        return name or author_id
    except Exception:
        return author_id

def get_author_eids(author_id: str) -> List[str]:
    try:
        s = ScopusSearch(f"AU-ID({author_id})", subscriber=True)
        return s.get_eids() or []
    except Exception as e:
        warnings.warn(f"ScopusSearch failed for AU-ID({author_id}): {e}")
        return []

def _safe_int(v) -> Optional[int]:
    try:
        return int(v)
    except Exception:
        return None

def get_article_metadata(eid: str, target_auid: Optional[str] = None):
    """
    Return article metadata dict. Also detect whether the given target_auid
    appears in the article and whether that author entry has affiliation_id == AFF_ID.
    """
    try:
        ar = AbstractRetrieval(eid, view="FULL")
    except ScopusException as e:
        warnings.warn(f"AbstractRetrieval failed for {eid}: {e}")
        return None

    if ar.subtype not in ("ar", "re"):
        return None

    year = ""
    if getattr(ar, "coverDate", None):
        year = _s(ar.coverDate)[:4]

    issn_print, issn_elec = _extract_issns(ar)
    asjc_codes_csv, asjc_areas_csv, asjc_abbrevs_csv, _codes_set = _extract_asjc(ar)

    bau_flag = False
    bau_orgs = []
    auid_org_map: Dict[str, str] = {}
    auth_err = None
    groups = getattr(ar, "authorgroup", None)
    if groups:
        try:
            for g in groups:
                try:
                    g_auid = str(getattr(g, "auid", "") or "")
                    org_val = getattr(g, "organization", None)
                    aff_id_val = _safe_int(getattr(g, "affiliation_id", None))
                    if g_auid:
                        auid_org_map[g_auid] = _s(org_val)
                    if target_auid and g_auid == str(target_auid) and AFF_ID and aff_id_val == _safe_int(AFF_ID):
                        bau_flag = True
                        if org_val:
                            bau_orgs.append(_s(org_val))
                except Exception:
                    continue
        except Exception as ex:
            auth_err = str(ex)
    else:
        try:
            authors = getattr(ar, "authors", None) or []
            for a in authors:
                try:
                    g_auid = str(getattr(a, "auid", "") or "")
                    org_val = getattr(a, "orgname", None) or getattr(a, "affiliation", None) or ""
                    if g_auid:
                        auid_org_map[g_auid] = _s(org_val)
                except Exception:
                    continue
        except Exception:
            pass

    insts = getattr(ar, "affiliation", None) or []

    return {
        "eid": _s(eid),
        "title": _s(ar.title),
        "year": _s(year),
        "publication_name": _s(getattr(ar, "publicationName", "")),
        "subtype": _s(ar.subtype),
        "doi": _s(getattr(ar, "doi", "")),
        "source_id": _s(getattr(ar, "source_id", "")),
        "issn_print": _s(issn_print),
        "issn_electronic": _s(issn_elec),
        "asjc_codes": asjc_codes_csv,
        "asjc_areas": asjc_areas_csv,
        "asjc_abbrevs": asjc_abbrevs_csv,
        "authors_count": len(ar.authors) if getattr(ar, "authors", None) else 1,
        "combined": "; ".join([t for t in (getattr(ar, "authkeywords", []) or []) if _s(t)]),
        "abstract": _s(getattr(ar, "description", "")),
        "author_org_map_json": json.dumps(auid_org_map, ensure_ascii=False),
        "is_bau_author": bau_flag,
        "bau_author_orgs": "; ".join(dict.fromkeys(bau_orgs)),
        "affiliations_list": [(getattr(i, "name", "") if hasattr(i, "name") else _s(i)) for i in insts],
        "auth_group_error": auth_err,
    }

# ----------------------- APP calculation helpers -----------------------

def _qc_from_percentile(p: Optional[float]) -> Optional[float]:
    if p is None:
        return None
    try:
        p = float(p)
    except Exception:
        return None
    if p >= 90:   # Top 10%
        return 1.4
    if p >= 75:   # Q1
        return 1.0
    if p >= 50:   # Q2
        return 0.8
    if p >= 25:   # Q3
        return 0.6
    if p >= 0:    # Q4
        return 0.4
    return None

def _ac_from_authors(n: Any) -> float:
    try:
        n = int(n)
    except Exception:
        n = 1
    return 1.2 if n <= 1 else 1.2 / max(n, 1)

def _to_int_year(y: Any) -> Optional[int]:
    try:
        s = str(y)
        m = re.search(r"\d{4}", s)
        return int(m.group(0)) if m else None
    except Exception:
        return None

def build_app_sheet(df_articles: pd.DataFrame, now_year: Optional[int] = None) -> Tuple[pd.DataFrame, Dict[str, Any]]:
    """
    Build APP sheet and summary.
    NOTE: Years considered are fixed to 2022, 2023, 2024 as requested.
    Only subtype == 'ar' (article) are eligible.
    APP totals and eligibility are described per academic year (AY).
    """
    if df_articles is None or df_articles.empty:
        return pd.DataFrame(), {"app_total": 0.0, "eligibility": "No eligible items", "years": []}

    # Fixed window per user's request
    years_ok = {2022, 2023, 2024}

    tmp = df_articles.copy()
    tmp["year_i"] = tmp.get("year", "").apply(_to_int_year)
    tmp["cs_percentile_num"] = pd.to_numeric(tmp.get("cs_percentile"), errors="coerce")
    tmp["authors_count_i"] = pd.to_numeric(tmp.get("authors_count"), errors="coerce").fillna(1).astype(int)
    tmp["subtype_norm"] = tmp.get("subtype", "").astype(str).str.lower()

    eligible = tmp[
        tmp["year_i"].isin(years_ok) &
        tmp["cs_percentile_num"].notna() &
        (tmp["subtype_norm"] == "ar")
    ].copy()

    if eligible.empty:
        return pd.DataFrame(), {"app_total": 0.0, "eligibility": "No eligible items (no 'ar' articles in 2022-2024)", "years": sorted(years_ok)}

    eligible["QC"] = eligible["cs_percentile_num"].apply(_qc_from_percentile)
    eligible["AC"] = eligible["authors_count_i"].apply(_ac_from_authors)
    eligible = eligible[eligible["QC"].notna()].copy()
    if eligible.empty:
        return pd.DataFrame(), {"app_total": 0.0, "eligibility": "No eligible items (QC missing)", "years": sorted(years_ok)}

    eligible["Contribution"] = eligible["AC"] * eligible["QC"]
    eligible["AC"] = eligible["AC"].round(2)
    eligible["QC"] = eligible["QC"].round(2)
    eligible["Contribution"] = eligible["Contribution"].round(2)

    out_cols = [
        "eid", "title", "year", "publication_name",
        "authors_count", "cs_percentile", "quartile",
        "AC", "QC", "Contribution"
    ]
    for c in out_cols:
        if c not in eligible.columns:
            eligible[c] = pd.Series(dtype="object")
    df_app = eligible[out_cols].sort_values(["year", "Contribution"], ascending=[False, False]).reset_index(drop=True)

    app_total = float(df_app["Contribution"].sum().round(2))

    # Eligibility messages explicitly mention "per academic year (AY)"
    if app_total > 1.0:
        elig = "APP > 1.0 → up to 2 supports per academic year (AY) (only 1 requires full indexing & APP check)"
    elif app_total >= 0.4:
        elig = "0.4 ≤ APP ≤ 1.0 → 1 support per academic year (AY)"
    else:
        elig = "APP < 0.4 → 1 support per academic year (AY) (if other criteria met)"

    summary = {"app_total": round(app_total, 2), "eligibility": elig, "years": sorted(years_ok)}
    return df_app, summary

# ----------------------- Excel writer (Articles / BAU Articles / APP) -----------------

def _write_excel_xlsxwriter(articles_df: pd.DataFrame, bau_df: pd.DataFrame, path: Path, app_df: Optional[pd.DataFrame] = None, app_summary: Optional[Dict[str, Any]] = None):
    with pd.ExcelWriter(path, engine="xlsxwriter") as xl:
        # Articles (ALL)
        articles_df.to_excel(xl, sheet_name="Articles", index=False)
        ws = xl.sheets["Articles"]
        try:
            ws.freeze_panes(1, 0)
            ws.set_column("A:A", 20)  # eid
            ws.set_column("B:B", 50)  # title
            ws.set_column("C:C", 8)   # year
            ws.set_column("D:D", 36)  # publication_name
            ws.set_column("E:E", 8)   # subtype
            ws.set_column("F:F", 26)  # doi
            ws.set_column("G:G", 14)  # source_id
            ws.set_column("H:I", 14)  # ISSNs
            ws.set_column("J:J", 18)  # asjc_codes
            ws.set_column("K:K", 26)  # asjc_areas
            ws.set_column("L:L", 14)  # asjc_abbrevs
            ws.set_column("M:N", 14)  # cs_percentile / citescore
            ws.set_column("O:O", 10)  # quartile
            ws.set_column("P:P", 12)  # authors_count
            ws.set_column("Q:Q", 28)  # combined
            ws.set_column("R:R", 80)  # abstract
            ws.set_column("S:U", 28)  # affiliation fields
        except Exception:
            pass

        # BAU Articles
        bau_df.to_excel(xl, sheet_name="BAU Articles", index=False)
        ws2 = xl.sheets["BAU Articles"]
        try:
            ws2.freeze_panes(1, 0)
            ws2.set_column("A:A", 20)
            ws2.set_column("B:B", 50)
            ws2.set_column("C:C", 8)
            ws2.set_column("D:D", 36)
            ws2.set_column("E:E", 8)
            ws2.set_column("F:F", 26)
            ws2.set_column("G:G", 14)
            ws2.set_column("H:I", 14)
            ws2.set_column("J:J", 18)
            ws2.set_column("K:K", 26)
            ws2.set_column("L:L", 14)
            ws2.set_column("M:N", 14)
            ws2.set_column("O:O", 10)
            ws2.set_column("P:P", 12)
            ws2.set_column("Q:Q", 28)
            ws2.set_column("R:R", 80)
            ws2.set_column("S:U", 28)
        except Exception:
            pass

        # APP sheet (uses BAU Articles)
        if app_df is not None and not app_df.empty:
            ws3 = xl.book.add_worksheet("APP")
            xl.sheets["APP"] = ws3
            try:
                ws3.write(0, 0, "APP calculation — fixed years: 2022, 2023, 2024 (journal articles only; subtype == 'ar')")
                ws3.write(1, 0, "NOTE: APP totals and support thresholds are shown per academic year (AY).")
                if app_summary:
                    ws3.write(2, 0, "Years considered")
                    ws3.write(2, 1, ", ".join(str(y) for y in app_summary.get("years", [])))
                    ws3.write(3, 0, "APP Score (sum of AC×QC, rounded)")
                    ws3.write(3, 1, app_summary.get("app_total", 0.0))
                    ws3.write(4, 0, "Eligibility (per academic year)")
                    ws3.write(4, 1, app_summary.get("eligibility", ""))
                (app_df.reset_index(drop=True)).to_excel(xl, sheet_name="APP", index=False, startrow=6)
                ws3.freeze_panes(7, 0)
                ws3.set_column("A:A", 20)
                ws3.set_column("B:B", 60)
                ws3.set_column("C:C", 8)
                ws3.set_column("D:D", 36)
                ws3.set_column("E:E", 12)
                ws3.set_column("F:F", 14)
                ws3.set_column("G:G", 10)
                ws3.set_column("H:J", 14)
            except Exception:
                try:
                    app_df.to_excel(xl, sheet_name="APP", index=False)
                except Exception:
                    pass

def _write_excel_openpyxl(articles_df: pd.DataFrame, bau_df: pd.DataFrame, path: Path, app_df: Optional[pd.DataFrame] = None, app_summary: Optional[Dict[str, Any]] = None):
    with pd.ExcelWriter(path, engine="openpyxl") as xl:
        articles_df.to_excel(xl, sheet_name="Articles", index=False)
        bau_df.to_excel(xl, sheet_name="BAU Articles", index=False)
        try:
            if app_df is not None and not app_df.empty:
                app_df.to_excel(xl, sheet_name="APP", index=False, startrow=5)
        except Exception:
            try:
                app_df.to_excel(xl, sheet_name="APP", index=False)
            except Exception:
                pass

def save_author_excel(author_id: str, author_name: str, articles_df: pd.DataFrame, bau_df: pd.DataFrame, out_dir: Path) -> str:
    out_dir.mkdir(parents=True, exist_ok=True)
    path = out_dir / make_author_filename(author_name, author_id)

    need = [
        "eid","title","year","publication_name","subtype","doi",
        "source_id","issn_print","issn_electronic",
        "asjc_codes","asjc_areas","asjc_abbrevs",
        "cs_percentile","citescore","quartile",
        "authors_count","combined","abstract",
        "is_bau_author","bau_author_orgs","author_org_map_json"
    ]

    # Ensure columns present for both frames
    art = articles_df.copy() if articles_df is not None else pd.DataFrame()
    bau = bau_df.copy() if bau_df is not None else pd.DataFrame()
    for df in (art, bau):
        for c in need:
            if c not in df.columns:
                df[c] = pd.Series(dtype="object")
    art = art[need]
    bau = bau[need]

    # APP computed from BAU articles
    app_df, app_summary = build_app_sheet(bau)

    try:
        _write_excel_xlsxwriter(art, bau, path, app_df=app_df if not app_df.empty else None, app_summary=app_summary)
    except Exception:
        _write_excel_openpyxl(art, bau, path, app_df=app_df if not app_df.empty else None, app_summary=app_summary)
    return str(path)

# ----------------------- Orchestration --------------------------------

def process_author(author_id: str, cs_table: pd.DataFrame, out_dir: Path, sleep: float, serial_sleep: float, cs_by_source: Optional[pd.DataFrame] = None) -> Optional[str]:
    try:
        name = get_author_name(author_id)
        print(f"→ AU-ID {author_id} — {name} (AFF_ID filter: {AFF_ID})")
        eids = get_author_eids(author_id)
        recs_all: List[Dict[str, Any]] = []
        recs_bau: List[Dict[str, Any]] = []

        for eid in eids:
            md = get_article_metadata(eid, target_auid=author_id)
            if md:
                rec = {
                    "eid": md.get("eid"),
                    "title": md.get("title"),
                    "year": md.get("year"),
                    "publication_name": md.get("publication_name"),
                    "subtype": md.get("subtype"),
                    "doi": md.get("doi"),
                    "source_id": md.get("source_id"),
                    "issn_print": md.get("issn_print"),
                    "issn_electronic": md.get("issn_electronic"),
                    "asjc_codes": md.get("asjc_codes"),
                    "asjc_areas": md.get("asjc_areas"),
                    "asjc_abbrevs": md.get("asjc_abbrevs"),
                    "authors_count": md.get("authors_count"),
                    "combined": md.get("combined"),
                    "abstract": md.get("abstract"),
                    "is_bau_author": md.get("is_bau_author"),
                    "bau_author_orgs": md.get("bau_author_orgs"),
                    "author_org_map_json": md.get("author_org_map_json"),
                }
                # add to "all articles" unconditionally
                recs_all.append(rec)
                # add to BAU-only if author-affiliated on this record
                if md.get("is_bau_author", False):
                    recs_bau.append(rec)
            if sleep:
                time.sleep(sleep)

        df_all = pd.DataFrame(recs_all)
        df_bau = pd.DataFrame(recs_bau)

        if not df_all.empty:
            if cs_by_source is None:
                cs_by_source = build_cs_by_source(cs_table, serial_sleep=serial_sleep)
            df_all = enrich_with_citescore_sourceid_asjc(df_all, cs_table, cs_by_source)

        if not df_bau.empty:
            if cs_by_source is None:
                cs_by_source = build_cs_by_source(cs_table, serial_sleep=serial_sleep)
            df_bau = enrich_with_citescore_sourceid_asjc(df_bau, cs_table, cs_by_source)

        path = save_author_excel(author_id, name, df_all if not df_all.empty else pd.DataFrame(), df_bau if not df_bau.empty else pd.DataFrame(), out_dir)
        print(f"   ✓ Wrote {Path(path).name}  (Articles: {len(df_all) if df_all is not None else 0}, BAU Articles: {len(df_bau) if df_bau is not None else 0})")
        return path
    except Exception as e:
        print(f"   ⚠️ Failed {author_id}: {e}")
        return None

# ----------------------- Public API (Jupyter) -------------------------

def run(
    auids: Optional[str] = None,
    citescore: Optional[str] = None,
    outdir: str = "authors",
    sleep: float = 0.05,
    serial_sleep: float = 0.1,
    no_prompt: bool = True,
    aff_id: Optional[str] = None,
) -> None:
    global AFF_ID
    if aff_id is not None:
        AFF_ID = aff_id

    out_dir = Path(outdir)
    cs_path = resolve_citescore_path(citescore, no_prompt=no_prompt)
    if not cs_path:
        raise FileNotFoundError(
            "Could not locate CiteScore file. Pass citescore=..., set CITESCORE_CSV env var, or place the file in a default folder."
        )
    print(f"Using CiteScore file: {cs_path}")
    cs_table = load_citescore_table(cs_path)

    ids = read_auids_from_cli_or_file(auids)
    if not ids:
        print("No AU-IDs provided. Provide auids='555...,572...' or authors.txt file.")
        return
    print(f"CiteScore rows: {len(cs_table)} | Authors: {len(ids)} | Affiliation filter: {AFF_ID}")

    cs_by_source = build_cs_by_source(cs_table, serial_sleep=serial_sleep)

    out_dir.mkdir(parents=True, exist_ok=True)
    for i, auid in enumerate(ids, 1):
        print(f"[{i}/{len(ids)}]")
        process_author(auid, cs_table, out_dir, sleep=sleep, serial_sleep=serial_sleep, cs_by_source=cs_by_source)
    print(f"\nDone. Files saved to: {out_dir.resolve()}")

# ----------------------- CLI -----------------------------------------

def main(argv: Optional[list] = None):
    global AFF_ID
    ap = argparse.ArgumentParser(description="Fetch Scopus pubs by AU-ID and match CiteScore by Source ID then ASJC (ISSN fallback). Produces 'Articles', 'BAU Articles' and 'APP' sheets per author. APP thresholds/messages are per academic year (AY).")
    ap.add_argument("--auids", type=str, default=None, help="Comma/space-separated Scopus Author IDs. If omitted, reads authors.txt")
    ap.add_argument("--citescore", type=str, default=None, help="Path to CiteScore CSV/XLSX, or a folder containing it")
    ap.add_argument("--outdir", type=str, default="authors", help="Output directory (default: ./authors)")
    ap.add_argument("--sleep", type=float, default=0.05, help="Sleep between EID fetches (seconds)")
    ap.add_argument("--serial-sleep", type=float, default=0.1, help="Sleep between SerialTitle ISSN lookups (seconds)")
    ap.add_argument("--no-prompt", action="store_true", help="Do not prompt for missing CiteScore path; exit with error")
    ap.add_argument("--aff-id", type=str, default=None, help=f"Optional affiliation ID to isolate author-affiliated publications (default: {AFF_ID_DEFAULT})")
    args, _unknown = ap.parse_known_args(argv)

    if args.aff_id is not None:
        AFF_ID = args.aff_id

    cs_path = resolve_citescore_path(args.citescore, no_prompt=args.no_prompt)
    if not cs_path:
        if _is_ipython() and not args.no_prompt:
            try:
                entered = input("Enter CiteScore CSV/XLSX path or containing folder: ").strip('"').strip()
            except EOFError:
                entered = ""
            if entered:
                cs_path = resolve_citescore_path(entered, no_prompt=True)
        if not cs_path:
            print("❌ Could not locate the CiteScore file. Pass --citescore or set CITESCORE_CSV.")
            if _is_ipython():
                return
            sys.exit(2)
    print(f"Using CiteScore file: {cs_path}")
    cs_table = load_citescore_table(cs_path)

    auids = read_auids_from_cli_or_file(args.auids)
    if not auids and _is_ipython():
        try:
            entered = input("Enter Scopus Author IDs (comma or space separated), or leave blank to cancel: ").strip()
        except EOFError:
            entered = ""
        if entered:
            auids = read_auids_from_cli_or_file(entered)
    if not auids:
        print("No AU-IDs provided. Use --auids, or create authors.txt with one AU-ID per line.")
        if _is_ipython():
            return
        sys.exit(0)

    out_dir = Path(args.outdir)
    print(f"CiteScore rows: {len(cs_table)} | Authors: {len(auids)} | Out: {out_dir.resolve()} | Affiliation filter: {AFF_ID}")
    out_dir.mkdir(parents=True, exist_ok=True)

    cs_by_source = build_cs_by_source(cs_table, serial_sleep=args.serial_sleep)

    written = []
    for i, auid in enumerate(auids, 1):
        print(f"[{i}/{len(auids)}] Processing AU-ID {auid} … (AFF_ID={AFF_ID})")
        p = process_author(auid, cs_table, out_dir, sleep=args.sleep, serial_sleep=args.serial_sleep, cs_by_source=cs_by_source)
        if p:
            written.append(p)

    print(f"\nDone. Wrote {len(written)} file(s) to {out_dir.resolve()}.")

if __name__ == "__main__":
    main()


Using CiteScore file: C:\Users\yusef.atteyih\Desktop\Academic Research Unit\Yusef ATTEYIH\Data Solutions\Data Solutions 2.0\APP Calculation\CiteScore 2024\CiteScore 2024 annual values.csv


Enter Scopus Author IDs (comma or space separated), or leave blank to cancel:  57193254610


CiteScore rows: 75679 | Authors: 1 | Out: C:\Users\yusef.atteyih\Desktop\Academic Research Unit\Yusef ATTEYIH\Data Solutions\Data Solutions 2.0\APP Calculation\authors | Affiliation filter: 60021379
[1/1] Processing AU-ID 57193254610 … (AFF_ID=60021379)
→ AU-ID 57193254610 — Fadime İrem Doğan (AFF_ID filter: 60021379)
   ✓ Wrote _fadime_irem_dogan___57193254610.xlsx  (Articles: 5, BAU Articles: 3)

Done. Wrote 1 file(s) to C:\Users\yusef.atteyih\Desktop\Academic Research Unit\Yusef ATTEYIH\Data Solutions\Data Solutions 2.0\APP Calculation\authors.
